#  검색 성능 향상을 위한 기법 

- 쿼리 확장 (Query Expansion)

### **학습 목표:** 쿼리 확장(Query Expansion) 기법을 구현하고 성능 개선을 측정한다

--- 

## 환경 설정 및 준비

`(1) Env 환경변수`

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

`(2) 기본 라이브러리`

In [2]:
import os
from glob import glob

from pprint import pprint
import json

`(3) langfuase handler 설정`

In [3]:
from langfuse.langchain import CallbackHandler

# LangChain 콜백 핸들러 생성
langfuse_handler = CallbackHandler()

`(4) 벡터스토어 로드`

In [4]:
# 벡터 저장소 로드 
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

chroma_db = Chroma(
    collection_name="db_korean_cosine_metadata",
    embedding_function=embeddings,
    persist_directory="./chroma_db",
)

`(5) 백터 검색기 생성`

In [5]:
# 기본 retriever 초기화
chroma_k_retriever = chroma_db.as_retriever(
    search_kwargs={"k": 4}
)

query = "리비안의 사업 경쟁력은 어디서 나오나요?"
retrieved_docs = chroma_k_retriever.invoke(query)

for doc in retrieved_docs:
    print(f"{doc.page_content} [출처: {doc.metadata['source']}]")
    print("="*200)

[출처] 이 문서는 리비안에 대한 문서입니다.
----------------------------------
- **회사 유형:** 상장
- **거래소:** NASDAQ: RIVN
- **설립:** 2009년 6월, 플로리다 주 록ledge
- **설립자:** R. J. 스캐린지
- **본사:** 미국 캘리포니아 주 어바인
- **서비스 지역:** 북미
- **주요 인물:** R. J. 스캐린지 (CEO)
- **제품:** 전기 자동차, 배터리
- **생산량 (2023):** 57,232대
- **서비스:** 전기 자동차 충전, 자동차 보험
- **수익 (2023):** 44억 3천만 미국 달러
- **순이익 (2023):** -54억 미국 달러
- **총 자산 (2023):** 168억 미국 달러 [출처: data\리비안_KR.md]
[출처] 이 문서는 리비안에 대한 문서입니다.
----------------------------------
**시설**

- **Irvine, California:** 차량 엔지니어링 및 설계에 중점을 둔 본사.
- **Normal, Illinois:** 차량 부품을 생산하고 조립을 수행하는 제조 공장.
- **Plymouth, Michigan:** 차량 엔지니어링, 프로토타입 제작, 공급망 및 회계에 중점을 둡니다.
- **Palo Alto, California:** 소프트웨어 개발 및 엔지니어링에 중점을 둡니다.
- Carson, California 및 Woking, England에 추가 사무실이 있습니다.
- 애틀랜타 동쪽에 있는 새로운 50억 달러 규모의 배터리 및 조립 공장은 보류 중입니다.

**재정**

Rivian의 재무 성과는 상당한 수익 성장과 상당한 순손실로 특징지어집니다. [출처: data\리비안_KR.md]
[출처] 이 문서는 리비안에 대한 문서입니다.
----------------------------------
**재정**

Rivian의 재무 성과는 상당한 수익 성장과 상당한 순손실로 특징지어집니다

---

## 쿼리 확장 (Query Expansion)

1. **Query Reformulation**
    - LLM을 사용하여 원래 질문을 다른 형태로 재작성하는 방식임
    - 동의어 확장, 질문 명확화, 관련 키워드 추가 등 다양한 방식으로 쿼리를 변형함
    - 검색의 다양성과 정확도를 향상시키는 특징이 있음

1. **Multi Query** 
    - Retriever에 지정된 LLM을 활용하여 원본 쿼리를 확장하는 방법임
    - 하나의 질문에 대해 다양한 관점과 표현으로 여러 개의 쿼리를 자동 생성함
    - LLM의 생성 능력을 활용하여 검색의 다양성과 포괄성을 향상시키는 특징이 있음

1. **Decomposition** 
    - 복잡한 질문을 여러 개의 단순한 하위 질문으로 분해하는 LEAST-TO-MOST PROMPTING 전략을 사용함
    - 각각의 하위 질문에 대해 개별적으로 검색을 수행하여 더 정확한 답변을 도출함
    - 복잡한 질문을 체계적으로 해결하면서 검색의 정확도를 높이는 특징이 있음

1. **Step-Back Prompting**
    - 주어진 구체적인 질문에서 한 걸음 물러나 더 일반적인 개념이나 배경을 먼저 검색함
    - 더 넓은 맥락에서 점차 구체적인 답변으로 좁혀가는 방식을 사용함
    - 복잡한 질문에 대해 더 포괄적이고 정확한 답변을 제공하는 특징이 있음

1. **HyDE (Hypothetical Document Embedding)**
    - 주어진 질문에 대해 가상의 이상적인 답변 문서를 LLM으로 생성함
    - 생성된 가상 문서를 임베딩하여 이를 기반으로 실제 문서를 검색하는 방식임
    - 질문의 맥락을 더 잘 반영한 검색이 가능한 특징이 있음

### 1) **Query Reformulation** 

- **Query Reformulation**은 **LLM**을 활용해 원본 질문을 다양한 형태로 재구성
- **동의어 확장**과 **키워드 추가**를 통해 검색 쿼리의 범위를 확장
- 모호한 질문을 **명확하게 구체화**하여 검색 정확도 향상
- 하나의 질문에 대해 **다양한 변형 쿼리**를 생성하여 검색 커버리지 확대

<center>
<img src="ref/query_rewrite.png" alt="rag" align="center" border="0"  width="800" height=auto>
</center>


[출처] https://arxiv.org/abs/2305.14283

In [6]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# 쿼리 리포뮬레이션을 위한 프롬프트 템플릿 정의
reformulation_template = """다음 질문을 검색 성능을 향상시키기 위해 다시 작성해주세요:
[질문]
{question}

다음 방식으로 질문을 재작성하세요:
1. 동의어 추가
2. 더 구체적인 키워드 포함
3. 관련된 개념 확장

[재작성된 질문]
"""

# 프롬프트 템플릿 생성
prompt = ChatPromptTemplate.from_template(reformulation_template)

# LLM 모델 초기화
llm = ChatOpenAI(model='gpt-4.1-mini', temperature=0)

# 쿼리 리포뮬레이션 체인 생성
reformulation_chain = prompt | llm | StrOutputParser()

# 체인 실행
query = "리비안의 사업 경쟁력은 어디서 나오나요?"
reformulated_query = reformulation_chain.invoke({"question": query})

print(f"쿼리: {query}")
pprint(f"리포뮬레이션된 쿼리: \n{reformulated_query}")

쿼리: 리비안의 사업 경쟁력은 어디서 나오나요?
('리포뮬레이션된 쿼리: \n'
 '[재작성된 질문]  \n'
 '리비안(Rivian)의 핵심 사업 경쟁력과 차별화된 강점은 무엇이며, 전기차 시장 내에서의 기술 혁신, 공급망 관리, 그리고 지속 가능성 '
 '전략은 어떻게 작용하고 있나요?')


In [7]:
# 리포뮬레이션된 쿼리로 검색
retrieved_docs = chroma_k_retriever.invoke(reformulated_query)

for doc in retrieved_docs:
    print(f"{doc.page_content} [출처: {doc.metadata['source']}]")
    print("="*200)

[출처] 이 문서는 리비안에 대한 문서입니다.
----------------------------------
**재정**

Rivian의 재무 성과는 상당한 수익 성장과 상당한 순손실로 특징지어집니다.

| 연도 | 수익 (백만 USD) | 순이익 (백만 USD) | 총 자산 (백만 USD) |
| ---- | --------------- | ----------------- | ------------------ |
| 2020 | 0               | -1,018            | 4,602              |
| 2021 | 55              | -4,688            | 22,294             |
| 2022 | 1,658           | -6,752            | 17,876             |
| 2023 | 4,434           | -5,432            | 16,778             |

**최대 주주**

2023년 12월 현재 최대 주주는 Amazon, T. Rowe Price International, The Vanguard Group, BlackRock 및 Fidelity Investments였습니다.

**협력**

Rivian은 Alex Honnold, the Honnold Foundation, Casa Pueblo, Ewan McGregor, Charley Boorman, Yakima 및 MAXTRAX와 파트너십을 맺었습니다.

**소송** [출처: data\리비안_KR.md]
[출처] 이 문서는 리비안에 대한 문서입니다.
----------------------------------
**시설**

- **Irvine, California:** 차량 엔지니어링 및 설계에 중점을 둔 본사.
- **Normal, Illinois:** 차량 부품을 생산하고 조립을 수행하는 제조 공장.
- **Plymouth, Michigan:** 차량 엔지니어링,

In [8]:
# Runnable 객체로 변환하여 검색기 생성 (LCEL)
reformulation_retriever = reformulation_chain | chroma_k_retriever

# 쿼리 리포뮬레이션 검색기 실행
query = "리비안의 사업 경쟁력은 어디서 나오나요?"
retrieved_docs = reformulation_retriever.invoke({"question": query})

for doc in retrieved_docs:
    print(f"{doc.page_content} [출처: {doc.metadata['source']}]")
    print("="*200)

[출처] 이 문서는 리비안에 대한 문서입니다.
----------------------------------
**재정**

Rivian의 재무 성과는 상당한 수익 성장과 상당한 순손실로 특징지어집니다.

| 연도 | 수익 (백만 USD) | 순이익 (백만 USD) | 총 자산 (백만 USD) |
| ---- | --------------- | ----------------- | ------------------ |
| 2020 | 0               | -1,018            | 4,602              |
| 2021 | 55              | -4,688            | 22,294             |
| 2022 | 1,658           | -6,752            | 17,876             |
| 2023 | 4,434           | -5,432            | 16,778             |

**최대 주주**

2023년 12월 현재 최대 주주는 Amazon, T. Rowe Price International, The Vanguard Group, BlackRock 및 Fidelity Investments였습니다.

**협력**

Rivian은 Alex Honnold, the Honnold Foundation, Casa Pueblo, Ewan McGregor, Charley Boorman, Yakima 및 MAXTRAX와 파트너십을 맺었습니다.

**소송** [출처: data\리비안_KR.md]
[출처] 이 문서는 리비안에 대한 문서입니다.
----------------------------------
**시설**

- **Irvine, California:** 차량 엔지니어링 및 설계에 중점을 둔 본사.
- **Normal, Illinois:** 차량 부품을 생산하고 조립을 수행하는 제조 공장.
- **Plymouth, Michigan:** 차량 엔지니어링,

---
### **[실습 1]**

- 쿼리 리포뮬레이션 체인을 개선합니다. 
- langfuse Tracing에서 로그를 확인하고 쿼리 변환 과정을 이해합니다. 

In [9]:
# ========== 실습 1: Query Reformulation 개선 ==========

# (1) 개선된 쿼리 리포뮬레이션 프롬프트 템플릿
improved_reformulation_template = """당신은 정보 검색 전문가입니다. 다음 질문을 검색 성능을 최대화하기 위해 개선해주세요.

[원본 질문]
{question}

다음 기준에 따라 질문을 개선하세요:

1. **키워드 다양화**: 핵심 개념의 동의어, 유의어, 관련 용어를 포함
2. **구체성 향상**: 모호한 표현을 구체적이고 명확한 표현으로 변경
3. **맥락 확장**: 질문과 관련된 배경 정보나 연관 개념을 자연스럽게 포함
4. **검색 친화적 구조**: 검색 엔진이 이해하기 쉬운 구조로 재구성

[개선된 검색 질문]
"""

# (2) 개선된 체인 구성 (Langfuse 트레이싱 포함)
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# 개선된 프롬프트 템플릿 생성
improved_prompt = ChatPromptTemplate.from_template(improved_reformulation_template)

# LLM 모델 초기화 (Langfuse 콜백 포함)
improved_llm = ChatOpenAI(
    model='gpt-4.1-mini', 
    temperature=0.3,  # 창의성과 일관성의 균형
    callbacks=[langfuse_handler]  # Langfuse 트레이싱 활성화
)

# 개선된 쿼리 리포뮬레이션 체인 생성
improved_reformulation_chain = improved_prompt | improved_llm | StrOutputParser()

# (3) 기존 체인과 개선된 체인 비교 테스트
test_queries = [
    "리비안의 사업 경쟁력은 어디서 나오나요?",
    "테슬라의 미래 전망은?", 
    "전기차 시장의 주요 동향은?"
]

print("=" * 80)
print("🔍 Query Reformulation 개선 결과 비교")
print("=" * 80)

for i, query in enumerate(test_queries, 1):
    print(f"\n📌 테스트 {i}: {query}")
    print("-" * 60)
    
    # 기존 체인 결과
    original_result = reformulation_chain.invoke({"question": query})
    print(f"🔸 기존 체인 결과:\n{original_result}")
    
    # 개선된 체인 결과  
    improved_result = improved_reformulation_chain.invoke({"question": query})
    print(f"🔹 개선된 체인 결과:\n{improved_result}")
    print("-" * 60)

# (4) 검색 성능 비교
print("\n" + "=" * 80)
print("📊 검색 성능 비교 (첫 번째 쿼리)")
print("=" * 80)

query = test_queries[0]

# 기존 방식으로 검색
original_reformed = reformulation_chain.invoke({"question": query})
original_docs = chroma_k_retriever.invoke(original_reformed)

# 개선된 방식으로 검색
improved_reformed = improved_reformulation_chain.invoke({"question": query})
improved_docs = chroma_k_retriever.invoke(improved_reformed)

print(f"🔸 기존 리포뮬레이션 쿼리:\n{original_reformed}\n")
print("🔸 기존 방식 검색 결과:")
for j, doc in enumerate(original_docs[:2], 1):
    print(f"   {j}. {doc.page_content[:100]}... [출처: {doc.metadata['source']}]")

print(f"\n🔹 개선된 리포뮬레이션 쿼리:\n{improved_reformed}\n")
print("🔹 개선된 방식 검색 결과:")
for j, doc in enumerate(improved_docs[:2], 1):
    print(f"   {j}. {doc.page_content[:100]}... [출처: {doc.metadata['source']}]")

# (5) 개선된 검색기 생성 (LCEL)
improved_reformulation_retriever = improved_reformulation_chain | chroma_k_retriever

print(f"\n✅ 개선된 Query Reformulation 체인이 성공적으로 구현되었습니다!")
print(f"💡 Langfuse에서 트레이싱 로그를 확인하여 쿼리 변환 과정을 분석해보세요.")

🔍 Query Reformulation 개선 결과 비교

📌 테스트 1: 리비안의 사업 경쟁력은 어디서 나오나요?
------------------------------------------------------------
🔸 기존 체인 결과:
[재작성된 질문]  
리비안(Rivian)의 핵심 사업 경쟁력과 차별화된 강점은 무엇이며, 전기차 시장 내에서의 기술 혁신, 공급망 관리, 그리고 지속 가능성 전략이 경쟁력에 어떻게 기여하고 있나요?
🔹 개선된 체인 결과:
리비안(Rivian)의 전기차(EV) 시장 내 사업 경쟁력은 무엇이며, 기술 혁신, 제품 차별화, 공급망 관리, 그리고 지속 가능성 전략 측면에서 어떻게 발휘되고 있나요?
------------------------------------------------------------

📌 테스트 2: 테슬라의 미래 전망은?
------------------------------------------------------------
🔸 기존 체인 결과:
[재작성된 질문]  
테슬라(Tesla)의 장기적인 성장 가능성과 전기차 시장 내 경쟁력, 자율주행 기술 발전, 에너지 저장 솔루션 확장 등 미래 전망은 어떻게 평가되고 있나요?
🔹 개선된 체인 결과:
테슬라(Tesla)의 2024년 이후 전기차 시장 내 성장 전망, 신기술 개발 계획, 재무 실적 및 경쟁사 대비 위치에 대한 최신 분석과 전문가 의견은 무엇인가요?
------------------------------------------------------------

📌 테스트 3: 전기차 시장의 주요 동향은?
------------------------------------------------------------
🔸 기존 체인 결과:
[재작성된 질문]  
전기차(EV) 산업의 최신 시장 동향과 성장 요인, 주요 기술 발전, 소비자 수요 변화, 그리고 글로벌 경쟁 구도는 어떻게 전개되고 있나요?
🔹 개선된 체인 결과:
전기차(EV) 시장의 최신 주

### 2) **Multi Query** 

- **Multi Query**는 **Retriever의 LLM**을 사용해 단일 질문을 다수의 쿼리로 확장
- 원본 질문에 대해 **다양한 관점**과 **표현 방식**으로 쿼리 자동 생성
- **LLM의 생성 능력**을 활용해 검색 범위를 자연스럽게 확장
- 검색의 **다양성**과 **포괄성**이 향상되어 관련 문서 검색 확률 증가

`(1) MultiQueryRetriever 활용`

- https://python.langchain.com/docs/how_to/MultiQueryRetriever/

In [10]:
# 멀티 쿼리 생성
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model='gpt-4.1-mini',
    temperature=0.7,
)

# 기본 retriever를 이용한 멀티 쿼리 생성 
multi_query_retriever = MultiQueryRetriever.from_llm(
    retriever=chroma_k_retriever, llm=llm
)

query = "리비안의 사업 경쟁력은 어디서 나오나요?"
retrieved_docs = multi_query_retriever.invoke(query)

for doc in retrieved_docs:
    print(f"{doc.page_content} [출처: {doc.metadata['source']}]")
    print("="*200)

[출처] 이 문서는 리비안에 대한 문서입니다.
----------------------------------
**재정**

Rivian의 재무 성과는 상당한 수익 성장과 상당한 순손실로 특징지어집니다.

| 연도 | 수익 (백만 USD) | 순이익 (백만 USD) | 총 자산 (백만 USD) |
| ---- | --------------- | ----------------- | ------------------ |
| 2020 | 0               | -1,018            | 4,602              |
| 2021 | 55              | -4,688            | 22,294             |
| 2022 | 1,658           | -6,752            | 17,876             |
| 2023 | 4,434           | -5,432            | 16,778             |

**최대 주주**

2023년 12월 현재 최대 주주는 Amazon, T. Rowe Price International, The Vanguard Group, BlackRock 및 Fidelity Investments였습니다.

**협력**

Rivian은 Alex Honnold, the Honnold Foundation, Casa Pueblo, Ewan McGregor, Charley Boorman, Yakima 및 MAXTRAX와 파트너십을 맺었습니다.

**소송** [출처: data\리비안_KR.md]
[출처] 이 문서는 리비안에 대한 문서입니다.
----------------------------------
**소송**

- 2020년 7월, Tesla는 Rivian이 독점 정보를 훔치고 직원을 빼갔다고 주장하며 소송을 제기했습니다.
- 2021년 3월, Illinois Automobile Dealers Association은 Rivian과 Lucid Motors가

`(2) Custom Prompt 활용`

In [11]:
from typing import List

from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_core.output_parsers import BaseOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI


# 모델 초기화
llm = ChatOpenAI(model="gpt-4.1-mini")

# 출력 파서: LLM 결과를 질문 리스트로 변환
class LineListOutputParser(BaseOutputParser[List[str]]):
    """Output parser for a list of lines."""

    def parse(self, text: str) -> List[str]:
        """Split the text into lines and remove empty lines."""
        return [line.strip() for line in text.strip().split("\n") if line.strip()]
    

# 쿼리 생성 프롬프트
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""Generate three different versions of the given user question to retrieve relevant documents from a vector database. The goal is to reframe the question from various perspectives to overcome limitations of distance-based similarity search.

    The generated questions should have the following characteristics:
    1. Maintain the core intent of the original question but use different expressions or viewpoints.
    2. Include synonyms or related concepts where possible.
    3. Slightly broaden or narrow the scope of the question to potentially include diverse relevant information.

    Write each question on a new line and include only the questions.

    [Original question]
    {question}
    
    [Alternative questions]
    """,
)

# 멀티쿼리 체인 구성
multiquery_chain = QUERY_PROMPT | llm | LineListOutputParser()

# 테스트 쿼리 실행
query = "리비안의 사업 경쟁력은 어디서 나오나요?"
result = multiquery_chain.invoke({"question": query})

print("생성된 대안 질문들:")
for i, q in enumerate(result, 1):
    print(f"{i}. {q}")

생성된 대안 질문들:
1. 리비안이 시장에서 경쟁 우위를 확보하는 주요 요인은 무엇인가요?
2. 리비안의 비즈니스 강점과 경쟁력의 근원은 어떻게 설명할 수 있나요?
3. 리비안이 산업 내에서 차별화되는 경쟁 요소들은 어떤 것들이 있나요?


In [12]:
# 다중 쿼리 검색기 생성
multi_query_custom_retriever = MultiQueryRetriever(
    retriever=chroma_k_retriever, # 기본 retriever
    llm_chain=multiquery_chain,   # 멀티쿼리 체인
    parser_key="lines"            # "lines": 출력 파서의 키
)  

retrieved_docs = multi_query_custom_retriever.invoke(query)

for doc in retrieved_docs:
    print(f"{doc.page_content} [출처: {doc.metadata['source']}]")
    print("="*200)

[출처] 이 문서는 리비안에 대한 문서입니다.
----------------------------------
- **회사 유형:** 상장
- **거래소:** NASDAQ: RIVN
- **설립:** 2009년 6월, 플로리다 주 록ledge
- **설립자:** R. J. 스캐린지
- **본사:** 미국 캘리포니아 주 어바인
- **서비스 지역:** 북미
- **주요 인물:** R. J. 스캐린지 (CEO)
- **제품:** 전기 자동차, 배터리
- **생산량 (2023):** 57,232대
- **서비스:** 전기 자동차 충전, 자동차 보험
- **수익 (2023):** 44억 3천만 미국 달러
- **순이익 (2023):** -54억 미국 달러
- **총 자산 (2023):** 168억 미국 달러 [출처: data\리비안_KR.md]
[출처] 이 문서는 리비안에 대한 문서입니다.
----------------------------------
**재정**

Rivian의 재무 성과는 상당한 수익 성장과 상당한 순손실로 특징지어집니다.

| 연도 | 수익 (백만 USD) | 순이익 (백만 USD) | 총 자산 (백만 USD) |
| ---- | --------------- | ----------------- | ------------------ |
| 2020 | 0               | -1,018            | 4,602              |
| 2021 | 55              | -4,688            | 22,294             |
| 2022 | 1,658           | -6,752            | 17,876             |
| 2023 | 4,434           | -5,432            | 16,778             |

**최대 주주**

2023년 12월 현재 최대 주주는 Amazon, T. Rowe Price International, The Vang

---
### **[실습 2]**

- 멀티쿼리 체인의 구조를 분석하고 개선합니다.  
- langfuse Tracing에서 로그를 확인하고 쿼리 변환 과정을 이해합니다. 

In [28]:
# ========== 실습 2: MultiQuery 체인 구조 분석 및 개선 ==========

# (1) 기존 MultiQuery 체인 구조 분석
print("=" * 80)
print("🔍 기존 MultiQuery 체인 구조 분석")
print("=" * 80)

# 기존 체인의 구성 요소 분석
print("📌 기존 MultiQuery 체인 구조:")
print("1. QUERY_PROMPT: 질문 생성을 위한 프롬프트 템플릿")
print("2. LLM: gpt-4.1-mini 모델")
print("3. LineListOutputParser: 출력을 리스트로 파싱")
print("4. MultiQueryRetriever: 다중 쿼리 기반 검색기")

# (2) 개선된 MultiQuery 프롬프트 설계
IMPROVED_MULTIQUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""당신은 정보 검색 전문가입니다. 주어진 질문에 대해 다양한 관점에서 5개의 검색 질문을 생성해주세요.

각 질문은 다음 기준을 충족해야 합니다:
1. **다양한 관점**: 기술적, 경제적, 시장적, 전략적 관점 등
2. **세분화된 접근**: 원본 질문의 다양한 측면을 다루도록 구성
3. **검색 최적화**: 벡터 검색에 효과적인 키워드와 구조 사용
4. **명확성**: 각 질문이 구체적이고 이해하기 쉽도록 작성
5. **포괄성**: 원본 질문의 모든 중요한 측면을 커버

생성 규칙:
- 각 질문은 서로 다른 관점이나 측면을 다뤄야 함
- 너무 유사한 질문은 피할 것
- 각 질문은 한 줄로 작성할 것
- 질문 번호나 추가 설명 없이 질문만 작성할 것

[원본 질문]
{question}

[생성된 다중 검색 질문들]
""",
)

# (3) 개선된 MultiQuery 체인 구성 (Langfuse 트레이싱 포함)
improved_multiquery_llm = ChatOpenAI(
    model="gpt-4.1-mini", 
    temperature=0.7,  # 다양성을 위해 온도 상승
    callbacks=[langfuse_handler]  # Langfuse 트레이싱 활성화
)

# 개선된 멀티쿼리 체인
improved_multiquery_chain = IMPROVED_MULTIQUERY_PROMPT | improved_multiquery_llm | LineListOutputParser()

# (4) 기존 vs 개선된 체인 비교 테스트
test_queries = [
    "리비안의 사업 경쟁력은 어디서 나오나요?",
    "테슬라의 자율주행 기술 수준은?",
    "전기차 배터리 기술의 미래는?"
]

print("\n" + "=" * 80)
print("📊 기존 vs 개선된 MultiQuery 체인 비교")
print("=" * 80)

for i, query in enumerate(test_queries, 1):
    print(f"\n🔸 테스트 {i}: {query}")
    print("-" * 60)
    
    # 기존 체인 결과
    try:
        original_queries = multiquery_chain.invoke({"question": query})
        print(f"🔹 기존 체인 생성 질문 ({len(original_queries)}개):")
        for j, q in enumerate(original_queries, 1):
            print(f"   {j}. {q}")
    except Exception as e:
        print(f"🔹 기존 체인 오류: {e}")
    
    # 개선된 체인 결과
    try:
        improved_queries = improved_multiquery_chain.invoke({"question": query})
        print(f"🔸 개선된 체인 생성 질문 ({len(improved_queries)}개):")
        for j, q in enumerate(improved_queries, 1):
            print(f"   {j}. {q}")
    except Exception as e:
        print(f"🔸 개선된 체인 오류: {e}")
    
    print("-" * 60)

# (5) 개선된 MultiQuery Retriever 생성
improved_multi_query_retriever = MultiQueryRetriever(
    retriever=chroma_k_retriever,
    llm_chain=improved_multiquery_chain,
    parser_key="lines"
)

# (6) 검색 성능 비교 테스트
print("\n" + "=" * 80)
print("🎯 검색 성능 비교 (첫 번째 테스트 쿼리)")
print("=" * 80)

test_query = test_queries[0]

# 기존 방식 검색
print("🔹 기존 MultiQuery 검색 결과:")
try:
    original_docs = multi_query_custom_retriever.invoke(test_query)
    print(f"검색된 문서 수: {len(original_docs)}")
    for i, doc in enumerate(original_docs[:3], 1):
        print(f"   {i}. {doc.page_content[:80]}... [출처: {doc.metadata['source']}]")
except Exception as e:
    print(f"검색 오류: {e}")

print("\n🔸 개선된 MultiQuery 검색 결과:")
try:
    improved_docs = improved_multi_query_retriever.invoke(test_query)
    print(f"검색된 문서 수: {len(improved_docs)}")
    for i, doc in enumerate(improved_docs[:3], 1):
        print(f"   {i}. {doc.page_content[:80]}... [출처: {doc.metadata['source']}]")
except Exception as e:
    print(f"검색 오류: {e}")

# (7) 질문 다양성 분석
print("\n" + "=" * 80)
print("📈 생성된 질문의 다양성 분석")
print("=" * 80)

def analyze_query_diversity(queries, title):
    """생성된 질문들의 다양성을 분석"""
    print(f"\n{title}")
    print(f"• 생성된 질문 수: {len(queries)}")
    
    # 키워드 추출 (간단한 분석)
    all_words = []
    for q in queries:
        words = q.lower().split()
        all_words.extend([w for w in words if len(w) > 2])
    
    unique_words = set(all_words)
    print(f"• 고유 키워드 수: {len(unique_words)}")
    print(f"• 평균 질문 길이: {sum(len(q) for q in queries) / len(queries):.1f}자")
    
    return len(unique_words), len(queries)

# 다양성 분석 실행
if 'original_queries' in locals():
    orig_diversity, orig_count = analyze_query_diversity(original_queries, "🔹 기존 체인 다양성:")

if 'improved_queries' in locals():
    impr_diversity, impr_count = analyze_query_diversity(improved_queries, "🔸 개선된 체인 다양성:")

# (8) 개선사항 요약
print("\n" + "=" * 80)
print("✅ MultiQuery 체인 개선사항 요약")
print("=" * 80)
print("1. 📝 더 구체적이고 체계적인 프롬프트 템플릿")
print("2. 🎯 5개의 다양한 관점 질문 생성 (기존 3개에서 확장)")
print("3. 🌡️ Temperature 0.7로 조정하여 다양성 증대")
print("4. 📊 Langfuse 트레이싱으로 질문 생성 과정 추적 가능")
print("5. 🔍 다양한 관점(기술적, 경제적, 시장적, 전략적) 포함")

print(f"\n💡 Langfuse에서 '{test_query}' 질문에 대한 다중 쿼리 생성 과정을 확인해보세요!")
print("🔗 트레이싱을 통해 각 단계별 실행 시간과 토큰 사용량도 분석할 수 있습니다.")

🔍 기존 MultiQuery 체인 구조 분석
📌 기존 MultiQuery 체인 구조:
1. QUERY_PROMPT: 질문 생성을 위한 프롬프트 템플릿
2. LLM: gpt-4.1-mini 모델
3. LineListOutputParser: 출력을 리스트로 파싱
4. MultiQueryRetriever: 다중 쿼리 기반 검색기

📊 기존 vs 개선된 MultiQuery 체인 비교

🔸 테스트 1: 리비안의 사업 경쟁력은 어디서 나오나요?
------------------------------------------------------------
🔹 기존 체인 생성 질문 (3개):
   1. 리비안의 핵심 경쟁력은 어떤 요소들에서 비롯되나요?
   2. 리비안이 시장에서 차별화되는 강점이나 경쟁 우위는 무엇입니까?
   3. 리비안의 비즈니스 성공 요인과 경쟁력을 결정짓는 주요 기반은 무엇인지 알고 싶습니다.
🔸 개선된 체인 생성 질문 (5개):
   1. 리비안의 전기차 기술 혁신과 연구개발 역량은 어떻게 경쟁력에 기여하는가
   2. 리비안의 사업 모델과 수익성 구조가 전통 자동차 산업 대비 어떤 경제적 이점을 제공하는가
   3. 리비안이 글로벌 전기차 시장에서 점유율 확대를 위해 채택한 마케팅 및 유통 전략은 무엇인가
   4. 리비안의 공급망 관리 및 생산 능력이 사업 경쟁력에 미치는 영향과 주요 전략은 무엇인가
   5. 리비안이 지속 가능성과 친환경 경영을 통해 시장 차별화를 이루는 방법은 무엇인가
------------------------------------------------------------

🔸 테스트 2: 테슬라의 자율주행 기술 수준은?
------------------------------------------------------------
🔹 기존 체인 생성 질문 (3개):
   1. 테슬라의 자동 운전 시스템의 현재 성능은 어떠한가?
   2. 테슬라가 개발한 자율주행 기술의 발전 단계와 특징은 무엇인가?
   3. 테슬라 자율

### 3) **Decomposition** 

- **단계별 분해 전략**을 통해 복잡한 질문을 작은 단위로 나누어 처리함
- 각 하위 질문마다 **독립적인 검색 프로세스**를 진행하여 정확도를 향상시킴
- **LEAST-TO-MOST PROMPTING**을 활용하여 체계적인 문제 해결 방식을 구현함
- 복잡한 문제를 단순화하여 검색 효율성을 극대화하는 방법론

<center>
<img src="ref/query_decomposition.png" alt="rag" align="center" border="0"  width="800" height=auto>
</center>


[출처] https://arxiv.org/pdf/2205.10625

In [14]:
from langchain.prompts import PromptTemplate
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to decompose the given input question into multiple sub-questions. 
    The goal is to break down the input into a set of sub-problems/sub-questions that can be answered independently.

    Follow these guidelines to generate the sub-questions:
    1. Cover various aspects related to the core topic of the original question.
    2. Each sub-question should be specific, clear, and answerable independently.
    3. Ensure that the sub-questions collectively address all important aspects of the original question.
    4. Consider temporal aspects (past, present, future) where applicable.
    5. Formulate the questions in a direct and concise manner.

    [Input question] 
    {question}

    [Sub-questions (5)]
    """,
)

# 쿼리 생성 체인
decomposition_chain = QUERY_PROMPT | llm | LineListOutputParser()

# 테스트 쿼리 실행
query = "리비안의 사업 경쟁력은 어디서 나오나요?"
result = decomposition_chain.invoke({"question": query})

print("생성된 서브 질문들:")
for i, q in enumerate(result, 1):
    print(f"{i}. {q}")

생성된 서브 질문들:
1. 1. 리비안의 주요 제품과 서비스는 무엇인가요?
2. 2. 리비안이 보유한 기술적 강점은 어떤 것들이 있나요?
3. 3. 리비안의 시장 진입 전략과 목표 고객층은 어떻게 되나요?
4. 4. 리비안이 경쟁사 대비 우위를 지니는 요소는 무엇인가요?
5. 5. 리비안의 미래 성장 가능성과 계획은 어떻게 되나요?


In [15]:
# 다중 쿼리 검색기 생성
multi_query_decompostion_retriever = MultiQueryRetriever(
    retriever=chroma_k_retriever,    # 기본 retriever
    llm_chain=decomposition_chain,   # 서브 질문 생성 체인
    parser_key="lines"               # "lines": 출력 파서의 키
)  

retrieved_docs = multi_query_decompostion_retriever.invoke(query)

for doc in retrieved_docs:
    print(f"{doc.page_content} [출처: {doc.metadata['source']}]")
    print("="*200)

[출처] 이 문서는 리비안에 대한 문서입니다.
----------------------------------
Rivian Automotive, Inc.는 2009년에 설립된 미국의 전기 자동차 제조업체, 자동차 기술 및 야외 레크리에이션 회사입니다.

**주요 정보:** [출처: data\리비안_KR.md]
[출처] 이 문서는 리비안에 대한 문서입니다.
----------------------------------
**시설**

- **Irvine, California:** 차량 엔지니어링 및 설계에 중점을 둔 본사.
- **Normal, Illinois:** 차량 부품을 생산하고 조립을 수행하는 제조 공장.
- **Plymouth, Michigan:** 차량 엔지니어링, 프로토타입 제작, 공급망 및 회계에 중점을 둡니다.
- **Palo Alto, California:** 소프트웨어 개발 및 엔지니어링에 중점을 둡니다.
- Carson, California 및 Woking, England에 추가 사무실이 있습니다.
- 애틀랜타 동쪽에 있는 새로운 50억 달러 규모의 배터리 및 조립 공장은 보류 중입니다.

**재정**

Rivian의 재무 성과는 상당한 수익 성장과 상당한 순손실로 특징지어집니다. [출처: data\리비안_KR.md]
[출처] 이 문서는 리비안에 대한 문서입니다.
----------------------------------
- **회사 유형:** 상장
- **거래소:** NASDAQ: RIVN
- **설립:** 2009년 6월, 플로리다 주 록ledge
- **설립자:** R. J. 스캐린지
- **본사:** 미국 캘리포니아 주 어바인
- **서비스 지역:** 북미
- **주요 인물:** R. J. 스캐린지 (CEO)
- **제품:** 전기 자동차, 배터리
- **생산량 (2023):** 57,232대
- **서비스:** 전기 자동차 충전, 자동차 보험
- **수익 (2023):** 44억 3천만 미국 달러
- **순이익 (2023):** 

---
### **[실습 3]**

- 쿼리 분해 체인의 구조를 분석하고 개선합니다.  
- langfuse Tracing에서 로그를 확인하고 쿼리 변환 과정을 이해합니다. 

In [29]:
# ========== 실습 3: Query Decomposition 체인 구조 분석 및 개선 ==========

# (1) 기존 Decomposition 체인 구조 분석
print("=" * 80)
print("🔍 기존 Query Decomposition 체인 구조 분석")
print("=" * 80)

print("📌 기존 Decomposition 체인 구조:")
print("1. QUERY_PROMPT: 서브 질문 생성을 위한 프롬프트 템플릿")
print("2. LLM: 복잡한 질문을 5개 서브 질문으로 분해")
print("3. LineListOutputParser: 출력을 리스트로 파싱")
print("4. MultiQueryRetriever: 각 서브 질문으로 독립적 검색")

print("\n📊 LEAST-TO-MOST 전략의 핵심:")
print("• 복잡한 문제를 단순한 하위 문제로 분해")
print("• 각 하위 문제를 독립적으로 해결")
print("• 단계별 해결을 통한 체계적 접근")

# (2) 개선된 Decomposition 프롬프트 설계
IMPROVED_DECOMPOSITION_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""당신은 복잡한 비즈니스 질문을 체계적으로 분석하는 전문가입니다. 
주어진 질문을 LEAST-TO-MOST 전략에 따라 6개의 구체적인 서브 질문으로 분해해주세요.

분해 원칙:
1. **기초 정보**: 기본적인 사실과 배경 정보 파악
2. **현재 상황**: 현재 상태와 최근 동향 분석
3. **핵심 요소**: 주요 구성 요소와 특징 분석
4. **경쟁 환경**: 시장 상황과 경쟁사 비교
5. **미래 전망**: 향후 계획과 예상되는 변화
6. **종합 평가**: 전반적인 평가와 결론

각 서브 질문은:
- 독립적으로 답변 가능해야 함
- 구체적이고 명확해야 함
- 원본 질문의 중요한 측면을 다뤄야 함
- 검색하기 쉬운 형태로 구성해야 함
- 서로 다른 관점이나 시간대를 다뤄야 함

[원본 질문]
{question}

[분해된 서브 질문들 (6개)]
""",
)

# (3) 개선된 Decomposition 체인 구성 (Langfuse 트레이싱 포함)
improved_decomposition_llm = ChatOpenAI(
    model="gpt-4.1-mini", 
    temperature=0.5,  # 구조화된 분해를 위해 중간 온도
    callbacks=[langfuse_handler]  # Langfuse 트레이싱 활성화
)

# 개선된 분해 체인
improved_decomposition_chain = IMPROVED_DECOMPOSITION_PROMPT | improved_decomposition_llm | LineListOutputParser()

# (4) 기존 vs 개선된 체인 비교 테스트
test_queries = [
    "리비안의 사업 경쟁력은 어디서 나오나요?",
    "테슬라의 글로벌 시장 전략은 무엇인가요?",
    "전기차 산업의 미래 성장 가능성은?"
]

print("\n" + "=" * 80)
print("📊 기존 vs 개선된 Decomposition 체인 비교")
print("=" * 80)

decomposition_results = {}

for i, query in enumerate(test_queries, 1):
    print(f"\n🔸 테스트 {i}: {query}")
    print("-" * 60)
    
    # 기존 체인 결과
    try:
        original_subqueries = decomposition_chain.invoke({"question": query})
        print(f"🔹 기존 체인 서브 질문 ({len(original_subqueries)}개):")
        for j, q in enumerate(original_subqueries, 1):
            print(f"   {j}. {q}")
        decomposition_results[f"original_{i}"] = original_subqueries
    except Exception as e:
        print(f"🔹 기존 체인 오류: {e}")
    
    # 개선된 체인 결과
    try:
        improved_subqueries = improved_decomposition_chain.invoke({"question": query})
        print(f"🔸 개선된 체인 서브 질문 ({len(improved_subqueries)}개):")
        for j, q in enumerate(improved_subqueries, 1):
            print(f"   {j}. {q}")
        decomposition_results[f"improved_{i}"] = improved_subqueries
    except Exception as e:
        print(f"🔸 개선된 체인 오류: {e}")
    
    print("-" * 60)

# (5) 개선된 Decomposition Retriever 생성
improved_decomposition_retriever = MultiQueryRetriever(
    retriever=chroma_k_retriever,
    llm_chain=improved_decomposition_chain,
    parser_key="lines"
)

# (6) 서브 질문별 검색 성능 분석
print("\n" + "=" * 80)
print("🎯 서브 질문별 검색 성능 분석")
print("=" * 80)

test_query = test_queries[0]

def analyze_subquery_search(subqueries, title, retriever):
    """각 서브 질문별 검색 결과 분석"""
    print(f"\n{title}")
    
    all_docs = []
    unique_sources = set()
    
    for i, subq in enumerate(subqueries, 1):
        try:
            docs = retriever.invoke(subq)
            all_docs.extend(docs)
            
            print(f"   서브질문 {i}: {subq[:50]}...")
            print(f"   검색 결과: {len(docs)}개 문서")
            
            for doc in docs:
                unique_sources.add(doc.metadata['source'])
                
        except Exception as e:
            print(f"   서브질문 {i} 검색 오류: {e}")
    
    print(f"   📊 총 검색 문서: {len(all_docs)}개")
    print(f"   📚 고유 출처: {len(unique_sources)}개")
    
    return all_docs, unique_sources

# 기존 방식 분석
if 'original_subqueries' in locals():
    orig_docs, orig_sources = analyze_subquery_search(
        original_subqueries, "🔹 기존 체인 검색 분석:", chroma_k_retriever
    )

# 개선된 방식 분석
if 'improved_subqueries' in locals():
    impr_docs, impr_sources = analyze_subquery_search(
        improved_subqueries, "🔸 개선된 체인 검색 분석:", chroma_k_retriever
    )

# (7) 전체 검색 성능 비교
print("\n" + "=" * 80)
print("🏆 전체 검색 성능 비교")
print("=" * 80)

# 기존 방식 전체 검색
print("🔹 기존 Decomposition 전체 검색:")
try:
    original_final_docs = multi_query_decompostion_retriever.invoke(test_query)
    print(f"최종 검색 문서: {len(original_final_docs)}개")
    for i, doc in enumerate(original_final_docs[:3], 1):
        print(f"   {i}. {doc.page_content[:80]}... [출처: {doc.metadata['source']}]")
except Exception as e:
    print(f"검색 오류: {e}")

# 개선된 방식 전체 검색
print("\n🔸 개선된 Decomposition 전체 검색:")
try:
    improved_final_docs = improved_decomposition_retriever.invoke(test_query)
    print(f"최종 검색 문서: {len(improved_final_docs)}개")
    for i, doc in enumerate(improved_final_docs[:3], 1):
        print(f"   {i}. {doc.page_content[:80]}... [출처: {doc.metadata['source']}]")
except Exception as e:
    print(f"검색 오류: {e}")

# (8) 서브 질문 품질 평가
print("\n" + "=" * 80)
print("📈 서브 질문 품질 평가")
print("=" * 80)

def evaluate_subquery_quality(subqueries, title):
    """서브 질문의 품질을 평가"""
    print(f"\n{title}")
    
    # 질문 길이 분석
    lengths = [len(q) for q in subqueries]
    avg_length = sum(lengths) / len(lengths)
    
    # 키워드 다양성 분석
    all_words = []
    for q in subqueries:
        words = q.lower().split()
        all_words.extend([w for w in words if len(w) > 2])
    unique_words = len(set(all_words))
    
    # 질문 유형 분석 (의문사 기반)
    question_types = {'무엇': 0, '어떻게': 0, '왜': 0, '언제': 0, '어디서': 0, '누가': 0}
    for q in subqueries:
        for qtype in question_types:
            if qtype in q:
                question_types[qtype] += 1
    
    print(f"   📏 평균 질문 길이: {avg_length:.1f}자")
    print(f"   🔤 고유 키워드: {unique_words}개")
    print(f"   ❓ 질문 유형 분포: {sum(question_types.values())}개 의문사 사용")
    
    return avg_length, unique_words

# 품질 평가 실행
if 'original_subqueries' in locals():
    orig_len, orig_words = evaluate_subquery_quality(original_subqueries, "🔹 기존 체인 품질:")

if 'improved_subqueries' in locals():
    impr_len, impr_words = evaluate_subquery_quality(improved_subqueries, "🔸 개선된 체인 품질:")

# (9) 개선사항 요약
print("\n" + "=" * 80)
print("✅ Query Decomposition 체인 개선사항 요약")
print("=" * 80)
print("1. 📋 체계적인 6단계 분해 구조 (기초→현재→핵심→경쟁→미래→평가)")
print("2. 🎯 LEAST-TO-MOST 전략의 명확한 적용")
print("3. 🔍 각 서브 질문의 독립성과 구체성 강화")
print("4. 📊 Langfuse 트레이싱으로 분해 과정 추적")
print("5. 🏗️ 구조화된 접근으로 포괄적 정보 수집")
print("6. 📈 서브 질문 품질과 다양성 향상")

print(f"\n💡 Langfuse에서 '{test_query}' 질문의 분해 과정을 확인해보세요!")
print("🔗 각 서브 질문이 어떻게 생성되고 검색되는지 상세히 분석할 수 있습니다.")

🔍 기존 Query Decomposition 체인 구조 분석
📌 기존 Decomposition 체인 구조:
1. QUERY_PROMPT: 서브 질문 생성을 위한 프롬프트 템플릿
2. LLM: 복잡한 질문을 5개 서브 질문으로 분해
3. LineListOutputParser: 출력을 리스트로 파싱
4. MultiQueryRetriever: 각 서브 질문으로 독립적 검색

📊 LEAST-TO-MOST 전략의 핵심:
• 복잡한 문제를 단순한 하위 문제로 분해
• 각 하위 문제를 독립적으로 해결
• 단계별 해결을 통한 체계적 접근

📊 기존 vs 개선된 Decomposition 체인 비교

🔸 테스트 1: 리비안의 사업 경쟁력은 어디서 나오나요?
------------------------------------------------------------
🔹 기존 체인 서브 질문 (5개):
   1. 1. 리비안은 어떤 핵심 기술을 보유하고 있나요?
   2. 2. 리비안이 제공하는 제품과 서비스의 특징은 무엇인가요?
   3. 3. 리비안의 시장 점유율과 경쟁사와의 차별점은 무엇인가요?
   4. 4. 리비안의 공급망과 생산 역량은 어떻게 구성되어 있나요?
   5. 5. 리비안의 미래 성장 전략과 혁신 계획은 무엇인가요?
🔸 개선된 체인 서브 질문 (6개):
   1. 1. 리비안(Rivian)의 주요 사업 분야와 제품 라인업은 무엇인가요?
   2. 2. 현재 리비안의 시장 점유율과 최근 매출 및 생산 실적은 어떻게 되나요?
   3. 3. 리비안의 기술력, 생산 능력, 공급망 등 핵심 경쟁 요소들은 어떤 특징을 가지고 있나요?
   4. 4. 전기차 시장 내에서 리비안과 주요 경쟁사들의 경쟁 우위와 차별점은 무엇인가요?
   5. 5. 리비안이 앞으로 계획 중인 신제품 출시, 기술 개발, 시장 확장 전략은 무엇인가요?
   6. 6. 전반적으로 리비안의 사업 경쟁력을 종합적으로 평가할 때 강점과 약점은 무엇이며, 향후 성공 가능성은 어떻게 보이나요?
------

### 4) **Step-Back Prompting**

- **단계적 후퇴 방식**을 통해 구체적 질문을 일반적 맥락에서 접근함
- **맥락 기반 검색**으로 넓은 관점에서 구체적 답변으로 좁혀나감
- **포괄적 접근법**을 활용하여 복잡한 질문에 대한 이해도를 높임
- 일반적 맥락에서 시작하여 구체적 해답을 찾아가는 체계적 접근 방식

<center>
<img src="ref/query_stepback.png" alt="rag" align="center" border="0"  width="800" height=auto>
</center>

[출처] https://arxiv.org/pdf/2310.06117

`(1) Step-Back 질문 생성`

In [17]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

# Few Shot 예제 - (구체적 질문, 포괄적 질문) 쌍
examples = [
    {
        "input": "애플의 M1 칩 개발이 기업 가치에 미친 영향은?",
        "output": "기업의 핵심 기술 내재화가 경쟁우위에 미치는 영향은 무엇인가?",
    },
    {
        "input": "아마존의 AWS가 수익성에 기여하는 방식은?",
        "output": "기업의 새로운 사업 영역 확장이 수익 구조에 미치는 영향은 무엇인가?",
    },
    {
        "input": "토요타의 하이브리드 기술 전략의 핵심은?",
        "output": "자동차 산업에서 친환경 기술 혁신이 기업 성장에 미치는 영향은 무엇인가?",
    }
]

# 프롬프트 템플릿 초기화
example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{input}"),
    ("ai", "{output}"),
])

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

# Step-Back 생성을 위한 프롬프트
step_back_prompt = ChatPromptTemplate.from_messages([
            (
                "system",
                """당신은 기업 분석 전문가입니다. 특정 기업에 대한 구체적인 질문을 해당 산업이나 비즈니스 전반의 일반적인 관점에서 
                재해석하는 것이 임무입니다. 산업 동향, 경쟁 구도, 기술 혁신, 사업 모델 등의 관점에서 더 포괄적인 질문으로 
                바꾸어 주세요. 다음은 예시입니다:"""
            ),
            few_shot_prompt,
            ("user", "{question}"),
        ])

# Step-Back 체인 생성
step_back_chain = step_back_prompt | llm | StrOutputParser()

# Step-Back 질문 생성
query = "리비안의 사업 경쟁력은 어디서 나오나요?"
step_back_question = step_back_chain.invoke({"question": query})

print(f"쿼리: {query}")
print(f"Step-Back 질문: {step_back_question}")

쿼리: 리비안의 사업 경쟁력은 어디서 나오나요?
Step-Back 질문: 전기차 산업에서 신생 기업이 차별화된 경쟁우위를 확보하는 핵심 요소는 무엇인가?


In [18]:
# Step-Back 검색기 생성
step_back_retriever = step_back_chain | chroma_k_retriever

# Step-Back 검색 실행
retrieved_docs = step_back_retriever.invoke({"question": query})

for doc in retrieved_docs:
    print(f"{doc.page_content} [출처: {doc.metadata['source']}]")
    print("="*200)

[출처] 이 문서는 테슬라에 대한 문서입니다.
----------------------------------
## 비즈니스 전략

Tesla의 전략은 배터리 비용을 줄이기 위해 고가, 소량 차량으로 시작한 다음 더 저렴하고 대량 차량을 제공하는 것입니다. Tesla는 자동차의 하드웨어를 지속적으로 업데이트하고 웹사이트와 회사 소유 매장을 통해 직접 차량을 판매합니다. Tesla는 수직적으로 통합되어 많은 구성 요소를 자체 개발합니다. Tesla는 일반적으로 지속 가능한 에너지 채택을 촉진하기 위해 경쟁 업체가 자사 기술을 라이선스하도록 허용합니다.

## 기술

### 배터리

Tesla는 CATL, LG Energy Solution 및 Panasonic에서 공급받은 원통형 및 각형 배터리 셀을 사용하고 있으며 자체 배터리를 생산할 수 있는 능력을 구축하고 있습니다. [출처: data\테슬라_KR.md]
[출처] 이 문서는 리비안에 대한 문서입니다.
----------------------------------
**시설**

- **Irvine, California:** 차량 엔지니어링 및 설계에 중점을 둔 본사.
- **Normal, Illinois:** 차량 부품을 생산하고 조립을 수행하는 제조 공장.
- **Plymouth, Michigan:** 차량 엔지니어링, 프로토타입 제작, 공급망 및 회계에 중점을 둡니다.
- **Palo Alto, California:** 소프트웨어 개발 및 엔지니어링에 중점을 둡니다.
- Carson, California 및 Woking, England에 추가 사무실이 있습니다.
- 애틀랜타 동쪽에 있는 새로운 50억 달러 규모의 배터리 및 조립 공장은 보류 중입니다.

**재정**

Rivian의 재무 성과는 상당한 수익 성장과 상당한 순손실로 특징지어집니다. [출처: data\리비안_KR.md]
[출처] 이 문서는 리비안에 대한 문서입니다.
----------------------------------
- R1T 배송은 2

`(2) 최종 답변 생성`

In [19]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate


# 프롬프트 템플릿 초기화
response_prompt = ChatPromptTemplate.from_template(
            """당신은 전문가입니다. 다음 컨텍스트와 질문을 바탕으로 포괄적인 답변을 제공해주세요.

            일반 컨텍스트:
            {normal_context}
            
            기본 개념 컨텍스트:
            {step_back_context}
            
            원래 질문: {question}
            
            답변:"""
        )

# 문서 포맷팅 함수
def format_docs(docs):
    return "\n".join([doc.page_content for doc in docs])


# 답변 생성 체인
answer_chain = (
            {
                "normal_context": chroma_k_retriever,
                "step_back_context": step_back_retriever,
                "question": RunnablePassthrough(),
            }
            | response_prompt
            | llm
            | StrOutputParser()
        )

# 답변 생성
query = "리비안의 사업 경쟁력은 어디서 나오나요?"
answer = answer_chain.invoke(query)

print(f"쿼리: {query}")
print(f"답변: {answer}")

쿼리: 리비안의 사업 경쟁력은 어디서 나오나요?
답변: 리비안(Rivian)의 사업 경쟁력은 여러 핵심 요소에서 비롯됩니다. 다음은 리비안의 경쟁력을 다각도로 분석한 내용입니다.

1. **전기차 분야 특화와 혁신적인 제품 라인업**  
   리비안은 전기 자동차와 배터리 생산에 집중하는 기업으로, 2023년 기준 약 57,000대의 전기차를 생산하며 전기차 시장에서 입지를 넓혀가고 있습니다. 특히 픽업트럭과 SUV 등 미국 시장 수요가 높은 세그먼트에 집중해 차별화된 제품을 제공하며, 북미 시장을 주요 서비스 지역으로 삼고 있다는 점이 경쟁력의 원천입니다.

2. **탄탄한 기술력 및 연구개발 인프라**  
   - **본사 및 핵심 연구 시설:** 어바인(Irvine, California)에 차량 엔지니어링 및 설계 부서를 운영하며, 플리머스(Plymouth, Michigan)와 팔로알토(Palo Alto, California)에서는 프로토타입 제작과 소프트웨어 개발에 집중하고 있습니다. 이는 차량 하드웨어와 소프트웨어를 통합하는 경쟁력으로 작용합니다.  
   - **배터리 및 조립 공장 투자:** 애틀랜타 동쪽에 50억 달러 규모의 배터리 및 조립 공장 건설을 추진 중이나 현재 보류 상태이나, 완공 시 생산 능력 확대와 원가 절감에서 큰 시너지 효과를 기대할 수 있습니다.

3. **강력한 재무 지원과 주요 투자자들**  
   리비안은 Amazon, T. Rowe Price International, The Vanguard Group, BlackRock, Fidelity Investments 등 세계적인 기관투자자들이 대주주로 참여하고 있어 자본 조달에 유리한 위치에 있습니다. 이는 대규모 설비 투자 및 연구개발에 필요한 자금을 안정적으로 확보할 수 있게 해줘 장기적인 성장 동력이 됩니다.

4. **생태계 구축과 협력 네트워크**  
   리비안은 전기차 충전 서비스와 자동차 보험 등 부가 서비스를 제공하며, Alex Honnold, Casa Pue

---
### **[실습 4]**

- Step back 체인의 구조를 분석하고 개선합니다.  
- langfuse Tracing에서 로그를 확인하고 쿼리 변환 과정을 이해합니다. 

In [30]:
# ========== 실습 4: Step-Back Prompting 체인 구조 분석 및 개선 ==========

# (1) 기존 Step-Back 체인 구조 분석
print("=" * 80)
print("🔍 기존 Step-Back Prompting 체인 구조 분석")
print("=" * 80)

print("📌 기존 Step-Back 체인 구조:")
print("1. Few-Shot Examples: 구체적 질문 → 포괄적 질문 변환 예시")
print("2. ChatPromptTemplate: 시스템 메시지와 예시를 포함한 프롬프트")
print("3. LLM: 구체적 질문을 일반적 맥락으로 확장")
print("4. 이중 검색: 일반 맥락 + 구체적 맥락 결합")

print("\n📊 Step-Back 전략의 핵심:")
print("• 구체적 질문에서 한 걸음 뒤로 물러나기")
print("• 더 넓은 맥락에서 접근하여 포괄적 이해")
print("• 일반적 원리와 구체적 사례 결합")

# (2) 개선된 Few-Shot 예제 설계
enhanced_examples = [
    {
        "input": "리비안의 전기차 배터리 성능은 어떤가요?",
        "output": "전기차 산업에서 배터리 기술이 기업 경쟁력과 시장 지위에 미치는 영향은 무엇인가?",
    },
    {
        "input": "테슬라의 자율주행 FSD 기술 수준은?",
        "output": "자동차 산업에서 자율주행 기술 개발이 기업의 미래 성장 전략에 미치는 영향은 무엇인가?",
    },
    {
        "input": "애플의 M1 칩 개발이 기업 가치에 미친 영향은?",
        "output": "기업의 핵심 기술 내재화가 경쟁우위와 생태계 구축에 미치는 영향은 무엇인가?",
    },
    {
        "input": "아마존의 AWS 클라우드 서비스 수익성은?",
        "output": "기업의 새로운 사업 영역 확장이 전체 수익 구조와 시장 지배력에 미치는 영향은 무엇인가?",
    },
    {
        "input": "구글의 AI 기술 투자 전략의 효과는?",
        "output": "기술 기업의 인공지능 투자가 장기적 경쟁우위와 시장 생태계에 미치는 영향은 무엇인가?",
    }
]

# (3) 개선된 Step-Back 프롬프트 구성
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

# 예시 프롬프트 템플릿
enhanced_example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{input}"),
    ("ai", "{output}"),
])

# Few-shot 프롬프트 생성
enhanced_few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=enhanced_example_prompt,
    examples=enhanced_examples,
)

# 개선된 Step-Back 프롬프트
enhanced_step_back_prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        """당신은 비즈니스 분석 및 전략 컨설팅 전문가입니다. 주어진 구체적인 기업이나 제품에 대한 질문을 
        해당 산업 전반이나 비즈니스 생태계의 일반적이고 포괄적인 관점에서 재해석하는 것이 임무입니다.

        변환 원칙:
        1. **산업 관점**: 개별 기업 → 산업 전반의 동향과 패턴
        2. **전략 관점**: 특정 기술/제품 → 경쟁 전략과 시장 지위
        3. **생태계 관점**: 단일 요소 → 전체 비즈니스 생태계
        4. **장기 관점**: 현재 상황 → 장기적 영향과 트렌드
        5. **원리 관점**: 구체적 사례 → 일반적 비즈니스 원리

        다음은 구체적 질문을 포괄적 관점으로 변환하는 예시입니다:"""
    ),
    enhanced_few_shot_prompt,
    (
        "user", 
        "다음 질문을 위의 원칙에 따라 더 포괄적이고 일반적인 관점의 질문으로 변환해주세요:\n\n{question}"
    ),
])

# (4) 개선된 Step-Back 체인 구성 (Langfuse 트레이싱 포함)
enhanced_step_back_llm = ChatOpenAI(
    model="gpt-4.1-mini", 
    temperature=0.3,  # 일관성 있는 변환을 위해 낮은 온도
    callbacks=[langfuse_handler]  # Langfuse 트레이싱 활성화
)

# 개선된 Step-Back 체인
enhanced_step_back_chain = enhanced_step_back_prompt | enhanced_step_back_llm | StrOutputParser()

# (5) 기존 vs 개선된 Step-Back 체인 비교
test_queries = [
    "리비안의 사업 경쟁력은 어디서 나오나요?",
    "테슬라의 슈퍼차저 네트워크 전략은?",
    "BYD의 글로벌 시장 확장 계획은?",
    "루시드 모터스의 기술적 우위는?"
]

print("\n" + "=" * 80)
print("📊 기존 vs 개선된 Step-Back 체인 비교")
print("=" * 80)

step_back_results = {}

for i, query in enumerate(test_queries, 1):
    print(f"\n🔸 테스트 {i}: {query}")
    print("-" * 60)
    
    # 기존 체인 결과
    try:
        original_step_back = step_back_chain.invoke({"question": query})
        print(f"🔹 기존 Step-Back 질문:\n   {original_step_back}")
        step_back_results[f"original_{i}"] = original_step_back
    except Exception as e:
        print(f"🔹 기존 체인 오류: {e}")
    
    # 개선된 체인 결과
    try:
        enhanced_step_back = enhanced_step_back_chain.invoke({"question": query})
        print(f"🔸 개선된 Step-Back 질문:\n   {enhanced_step_back}")
        step_back_results[f"enhanced_{i}"] = enhanced_step_back
    except Exception as e:
        print(f"🔸 개선된 체인 오류: {e}")
    
    print("-" * 60)

# (6) 개선된 Step-Back 검색기 구성
enhanced_step_back_retriever = enhanced_step_back_chain | chroma_k_retriever

# (7) 검색 성능 비교
print("\n" + "=" * 80)
print("🎯 검색 성능 비교 (첫 번째 테스트 쿼리)")
print("=" * 80)

test_query = test_queries[0]

# 기존 방식 검색
print("🔹 기존 Step-Back 검색 결과:")
try:
    original_docs = step_back_retriever.invoke({"question": test_query})
    print(f"검색된 문서 수: {len(original_docs)}")
    for i, doc in enumerate(original_docs[:3], 1):
        print(f"   {i}. {doc.page_content[:80]}... [출처: {doc.metadata['source']}]")
except Exception as e:
    print(f"검색 오류: {e}")

print("\n🔸 개선된 Step-Back 검색 결과:")
try:
    enhanced_docs = enhanced_step_back_retriever.invoke({"question": test_query})
    print(f"검색된 문서 수: {len(enhanced_docs)}")
    for i, doc in enumerate(enhanced_docs[:3], 1):
        print(f"   {i}. {doc.page_content[:80]}... [출처: {doc.metadata['source']}]")
except Exception as e:
    print(f"검색 오류: {e}")

# (8) 개선된 이중 맥락 답변 생성
enhanced_response_prompt = ChatPromptTemplate.from_template(
    """당신은 비즈니스 분석 전문가입니다. 다음 두 가지 컨텍스트를 종합하여 포괄적이고 통찰력 있는 답변을 제공해주세요.

**구체적 맥락 (직접 검색 결과):**
{normal_context}

**일반적 맥락 (Step-Back 검색 결과):**
{step_back_context}

**원본 질문:** {question}

답변 구성:
1. **일반적 원리**: Step-Back 맥락을 바탕으로 한 업계 전반의 원리와 트렌드
2. **구체적 분석**: 직접 검색 결과를 바탕으로 한 세부적 분석
3. **통합적 결론**: 두 관점을 종합한 최종 인사이트

**전문가 답변:**"""
)

# 개선된 답변 생성 체인
enhanced_answer_chain = (
    {
        "normal_context": lambda x: format_docs(chroma_k_retriever.invoke(x)),
        "step_back_context": lambda x: format_docs(enhanced_step_back_retriever.invoke({"question": x})),
        "question": RunnablePassthrough(),
    }
    | enhanced_response_prompt
    | ChatOpenAI(model="gpt-4.1-mini", temperature=0.2, callbacks=[langfuse_handler])
    | StrOutputParser()
)

# (9) 답변 품질 비교
print("\n" + "=" * 80)
print("📝 답변 품질 비교")
print("=" * 80)

# 기존 방식 답변
print("🔹 기존 Step-Back 답변:")
try:
    original_answer = answer_chain.invoke(test_query)
    print(f"{original_answer[:300]}...")
except Exception as e:
    print(f"답변 생성 오류: {e}")

print("\n🔸 개선된 Step-Back 답변:")
try:
    enhanced_answer = enhanced_answer_chain.invoke(test_query)
    print(f"{enhanced_answer[:300]}...")
except Exception as e:
    print(f"답변 생성 오류: {e}")

# (10) Step-Back 질문 품질 분석
print("\n" + "=" * 80)
print("📈 Step-Back 질문 품질 분석")
print("=" * 80)

def analyze_step_back_quality(original_q, step_back_q, title):
    """Step-Back 질문의 품질을 분석"""
    print(f"\n{title}")
    
    # 추상화 수준 분석
    original_words = set(original_q.lower().split())
    step_back_words = set(step_back_q.lower().split())
    
    # 키워드 변화 분석
    common_words = original_words.intersection(step_back_words)
    new_words = step_back_words - original_words
    
    print(f"   📏 원본 질문 길이: {len(original_q)}자")
    print(f"   📏 Step-Back 질문 길이: {len(step_back_q)}자")
    print(f"   🔤 공통 키워드: {len(common_words)}개")
    print(f"   ✨ 새로운 키워드: {len(new_words)}개")
    
    # 추상화 키워드 확인
    abstract_keywords = ['영향', '전략', '산업', '시장', '경쟁', '생태계', '원리', '동향', '패턴']
    abstract_count = sum(1 for keyword in abstract_keywords if keyword in step_back_q)
    print(f"   🎯 추상화 키워드: {abstract_count}개")
    
    return len(new_words), abstract_count

# 품질 분석 실행
if 'original_step_back' in locals() and 'enhanced_step_back' in locals():
    orig_new, orig_abstract = analyze_step_back_quality(
        test_query, original_step_back, "🔹 기존 Step-Back 품질:"
    )
    enh_new, enh_abstract = analyze_step_back_quality(
        test_query, enhanced_step_back, "🔸 개선된 Step-Back 품질:"
    )

# (11) 개선사항 요약
print("\n" + "=" * 80)
print("✅ Step-Back Prompting 체인 개선사항 요약")
print("=" * 80)
print("1. 📚 강화된 Few-Shot 예제 (5개 → 업계별 다양한 사례)")
print("2. 🎯 명확한 변환 원칙 (산업/전략/생태계/장기/원리 관점)")
print("3. 🔄 개선된 이중 맥락 결합 방식")
print("4. 📊 Langfuse 트레이싱으로 전체 과정 추적")
print("5. 🏗️ 구조화된 답변 생성 (일반원리→구체분석→통합결론)")
print("6. 📈 Step-Back 질문 품질 정량 분석")

print(f"\n💡 Langfuse에서 '{test_query}' 질문의 Step-Back 과정을 확인해보세요!")
print("🔗 구체적 질문이 어떻게 포괄적 관점으로 변환되는지 단계별로 분석할 수 있습니다.")
print("🎭 일반적 맥락과 구체적 맥락이 어떻게 결합되어 최종 답변이 생성되는지도 확인 가능합니다.")

🔍 기존 Step-Back Prompting 체인 구조 분석
📌 기존 Step-Back 체인 구조:
1. Few-Shot Examples: 구체적 질문 → 포괄적 질문 변환 예시
2. ChatPromptTemplate: 시스템 메시지와 예시를 포함한 프롬프트
3. LLM: 구체적 질문을 일반적 맥락으로 확장
4. 이중 검색: 일반 맥락 + 구체적 맥락 결합

📊 Step-Back 전략의 핵심:
• 구체적 질문에서 한 걸음 뒤로 물러나기
• 더 넓은 맥락에서 접근하여 포괄적 이해
• 일반적 원리와 구체적 사례 결합

📊 기존 vs 개선된 Step-Back 체인 비교

🔸 테스트 1: 리비안의 사업 경쟁력은 어디서 나오나요?
------------------------------------------------------------
🔹 기존 Step-Back 질문:
   신생 전기차 기업이 기존 완성차 업체와 차별화된 경쟁우위를 확보하는 전략적 요소는 무엇인가?
🔸 개선된 Step-Back 질문:
   전기차 산업에서 기업이 지속 가능한 경쟁우위를 확보하기 위한 핵심 요인들은 무엇인가?
------------------------------------------------------------

🔸 테스트 2: 테슬라의 슈퍼차저 네트워크 전략은?
------------------------------------------------------------
🔹 기존 Step-Back 질문:
   전기차 산업에서 충전 인프라 구축이 경쟁력 확보와 시장 확대에 미치는 영향은 무엇인가?
🔸 개선된 Step-Back 질문:
   전기차 산업에서 충전 인프라 구축 전략이 시장 경쟁력과 생태계 발전에 미치는 영향은 무엇인가?
------------------------------------------------------------

🔸 테스트 3: BYD의 글로벌 시장 확장 계획은?
----------------------------------------------------

### 5) **HyDE** (Hypothetical Document Embedding)

- **가상 문서 생성**을 통해 주어진 질문에 대해 가상의 이상적인 답변 문서를 LLM으로 생성함
- 생성된 문서의 **임베딩 기반 검색**으로 실제 문서와 매칭을 수행함
- **맥락 기반 검색 방식**으로 질문의 의도를 더 정확하게 반영함

<center>
<img src="ref/query_HyDE.png" alt="rag" align="center" border="0"  width="1000" height=auto>
</center>

[출처] https://arxiv.org/abs/2212.10496

`(1) 가상 문서 생성`

In [21]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

# HyDE를 위한 프롬프트 템플릿 생성
template = """주어진 질문에 대한 이상적인 문서 내용을 생성해주세요.
문서는 학술적이고 전문적인 톤으로 작성되어야 합니다.

질문: {question}

문서 내용:"""

hyde_prompt = ChatPromptTemplate.from_template(template)

# LLM 모델 초기화
hyde_llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

# 문서 생성 체인 생성
hyde_chain = hyde_prompt | hyde_llm | StrOutputParser()

# 문서 생성 실행
query = "리비안의 사업 경쟁력은 어디서 나오나요?"
hypothetical_doc = hyde_chain.invoke({"question": query})

print(f"쿼리: {query}")
print(f"문서 내용: {hypothetical_doc}")

쿼리: 리비안의 사업 경쟁력은 어디서 나오나요?
문서 내용: 리비안(Rivian)의 사업 경쟁력은 다각적인 요소에서 기인하며, 이는 전기차(EV) 시장 내에서의 차별화와 지속 가능한 성장 가능성을 뒷받침하는 핵심 동력으로 작용한다. 본 문서에서는 리비안의 경쟁력을 구성하는 주요 요인들을 기술적 혁신, 제품 포트폴리오, 공급망 관리, 브랜드 전략, 그리고 친환경 가치 중심 경영의 측면에서 분석하고자 한다.

첫째, 리비안은 첨단 전기차 플랫폼과 배터리 기술을 기반으로 한 기술적 혁신에서 경쟁우위를 확보하고 있다. 리비안의 독자적인 ‘스케이트보드’ 플랫폼은 배터리 팩, 전기 모터, 서스펜션, 제어 시스템을 통합하여 차량의 무게 중심을 낮추고 주행 안정성을 극대화한다. 또한, 고성능 배터리 관리 시스템(BMS)과 효율적인 에너지 회생 제동 시스템을 통해 주행 거리와 에너지 효율성을 향상시켜, 소비자 요구에 부합하는 실용적이고 신뢰성 높은 전기차를 제공한다.

둘째, 리비안은 SUV와 픽업트럭 등 미국 시장에서 수요가 높은 세그먼트에 집중한 제품 포트폴리오를 구축함으로써 시장 진입 장벽을 효과적으로 극복하고 있다. 특히, 전통적인 내연기관 차량이 강세인 픽업트럭 시장에서 전기차 모델을 선보임으로써 틈새시장을 공략하고 있으며, 이는 경쟁사 대비 차별화된 전략으로 평가된다. 더불어, 아웃도어 라이프스타일과 연계된 기능성 및 디자인 요소를 강조하여 특정 고객층의 충성도를 확보하고 있다.

셋째, 리비안은 공급망 관리와 생산 역량 강화에 주력하여 안정적인 제품 공급과 원가 경쟁력을 확보하고 있다. 초기 단계부터 주요 부품 공급업체와의 전략적 파트너십을 구축하고, 자체 생산 시설 확장에 투자함으로써 생산 병목 현상을 최소화하고 있다. 또한, 배터리 셀 제조 및 재활용 기술 개발에도 적극 참여하여 원재료 비용 절감과 친환경 생산 체계 구축에 기여하고 있다.

넷째, 브랜드 전략 측면에서 리비안은 혁신적이고 지속 가능한 모빌리티 솔루션 제공자로서의 이미지를 구축하고 있다. 친환경

`(2) 유사 문서 검색`

In [22]:
# 가상 문서를 기반으로 실제 문서 검색
    
retrieved_docs = chroma_k_retriever.invoke(hypothetical_doc)

for doc in retrieved_docs:
    print(f"{doc.page_content} [출처: {doc.metadata['source']}]")
    print("="*200)

[출처] 이 문서는 리비안에 대한 문서입니다.
----------------------------------
**시설**

- **Irvine, California:** 차량 엔지니어링 및 설계에 중점을 둔 본사.
- **Normal, Illinois:** 차량 부품을 생산하고 조립을 수행하는 제조 공장.
- **Plymouth, Michigan:** 차량 엔지니어링, 프로토타입 제작, 공급망 및 회계에 중점을 둡니다.
- **Palo Alto, California:** 소프트웨어 개발 및 엔지니어링에 중점을 둡니다.
- Carson, California 및 Woking, England에 추가 사무실이 있습니다.
- 애틀랜타 동쪽에 있는 새로운 50억 달러 규모의 배터리 및 조립 공장은 보류 중입니다.

**재정**

Rivian의 재무 성과는 상당한 수익 성장과 상당한 순손실로 특징지어집니다. [출처: data\리비안_KR.md]
[출처] 이 문서는 리비안에 대한 문서입니다.
----------------------------------
- **회사 유형:** 상장
- **거래소:** NASDAQ: RIVN
- **설립:** 2009년 6월, 플로리다 주 록ledge
- **설립자:** R. J. 스캐린지
- **본사:** 미국 캘리포니아 주 어바인
- **서비스 지역:** 북미
- **주요 인물:** R. J. 스캐린지 (CEO)
- **제품:** 전기 자동차, 배터리
- **생산량 (2023):** 57,232대
- **서비스:** 전기 자동차 충전, 자동차 보험
- **수익 (2023):** 44억 3천만 미국 달러
- **순이익 (2023):** -54억 미국 달러
- **총 자산 (2023):** 168억 미국 달러 [출처: data\리비안_KR.md]
[출처] 이 문서는 리비안에 대한 문서입니다.
----------------------------------
**재정**

Rivian의 재무 성과는 상당한 수익 성장과 상당한 순손실로 특징지어집니다

`(3) 최종 답변 생성`

In [23]:
# 최종 RAG를 위한 프롬프트 템플릿 생성
template = """다음 컨텍스트를 바탕으로 질문에 답변해주세요:

컨텍스트:
{context}

질문: {question}

답변:"""

rag_prompt =  ChatPromptTemplate.from_template(template)

# RAG 체인 생성
rag_llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)
rag_chain = rag_prompt | rag_llm | StrOutputParser()
    
# RAG 실행
query = "리비안의 사업 경쟁력은 어디서 나오나요?"
context = format_docs(retrieved_docs)

answer = rag_chain.invoke({"context": context, "question": query})

print(f"쿼리: {query}")
print(f"답변: {answer}")

쿼리: 리비안의 사업 경쟁력은 어디서 나오나요?
답변: 리비안의 사업 경쟁력은 다음과 같은 요소들에서 비롯됩니다:

1. **전기차 시장의 선도적 위치**  
   2021년 9월, 리비안은 완전 전기 픽업트럭 R1T를 소비자 시장에 최초로 출시하며 전기 픽업 분야에서 선도적인 입지를 확보했습니다. 이는 경쟁사 대비 빠른 시장 진입과 브랜드 인지도 확보에 유리하게 작용합니다.

2. **다양한 전문 시설과 기술 역량**  
   - 어바인 본사에서는 차량 엔지니어링 및 설계에 집중하여 혁신적인 제품 개발을 추진합니다.  
   - 노멀(일리노이) 제조 공장에서는 부품 생산과 조립을 담당해 생산 효율성을 높이고 있습니다.  
   - 플리머스(미시간)에서는 엔지니어링, 프로토타입 제작, 공급망 관리 및 회계 업무를 수행해 제품 개발과 운영의 통합적 관리를 지원합니다.  
   - 팔로알토(캘리포니아)에서는 소프트웨어 개발에 집중하여 차량의 스마트 기능과 사용자 경험을 강화합니다.  
   이러한 다각적인 기술 및 생산 인프라는 리비안이 전기차의 하드웨어와 소프트웨어를 통합적으로 개발할 수 있는 경쟁력을 제공합니다.

3. **강력한 투자자 및 파트너십 네트워크**  
   아마존, T. Rowe Price, Vanguard, BlackRock, Fidelity Investments 등 대형 투자자들이 최대 주주로 참여하고 있어 재무적 안정성과 성장 자본을 확보하고 있습니다.  
   또한, Alex Honnold, Casa Pueblo, Ewan McGregor 등 다양한 분야의 파트너와 협력하여 브랜드 가치와 사회적 책임을 강화하고 있습니다.

4. **제품 및 서비스의 다양성**  
   전기 자동차뿐만 아니라 배터리, 전기차 충전 서비스, 자동차 보험 등 관련 생태계를 구축하여 고객에게 통합적인 솔루션을 제공함으로써 경쟁 우위를 확보하고 있습니다.

5. **혁신과 성장에 대한 지속적인 투자**  
   50억 달러 규모의 배터리 및 조립 공장 건설 계획(

`(4) HyDE 체인 종합`

In [24]:
# Step 1. 가상 문서 생성
query = "테슬라의 경영진을 분석해주세요."

hypothetical_doc = hyde_chain.invoke({"question": query})

# Step 2. 유사 문서 검색
retrieved_docs = docs = chroma_k_retriever.invoke(hypothetical_doc)

# Step 3. 최종 답변 생성
final_answer = rag_chain.invoke(
    {
        "context": format_docs(retrieved_docs), 
        "question": query
    }
)

print(f"쿼리: {query}")
print(f"답변: {final_answer}")

쿼리: 테슬라의 경영진을 분석해주세요.
답변: 테슬라의 경영진은 회사 설립 초기부터 현재까지 주요 인물들이 중요한 역할을 해왔으며, 특히 다음과 같은 인물들이 핵심 경영진으로 자리잡고 있습니다.

1. **Elon Musk**  
   - 2004년 2월 시리즈 A 자금 조달을 주도하며 회장 겸 최대 주주가 되었고, 이후 CEO로서 테슬라의 전략적 방향과 혁신을 이끌고 있습니다.  
   - Musk는 테슬라의 비즈니스 전략, 기술 개발, 생산 확대, 그리고 홍보 및 대외 커뮤니케이션에 큰 영향을 미치고 있습니다.  
   - 다만, 그의 논란의 여지가 있는 발언과 행동으로 인해 소송 및 비판의 대상이 되기도 했습니다.

2. **Martin Eberhard와 Marc Tarpenning**  
   - 2003년 테슬라를 공동 창립했으며, 각각 초대 CEO와 CFO를 역임했습니다.  
   - 회사의 초기 비전과 설립 기반을 마련하는 데 중요한 역할을 했습니다.

3. **Ian Wright**  
   - 창립 초기 멤버로 합류하여 기술 및 제품 개발에 기여했습니다.

4. **J. B. Straubel**  
   - 2004년 CTO로 합류하여 테슬라의 기술 혁신, 특히 배터리 및 전기차 관련 핵심 기술 개발을 주도했습니다.

이외에도 테슬라는 수직적 통합과 자체 기술 개발을 강조하는 경영 방침을 통해 경영진이 기술과 생산, 판매 전반에 깊이 관여하는 구조를 갖추고 있습니다. 그러나 내부 고발자 보복, 근로자 권리 침해, 안전 결함 등과 관련된 소송과 논란도 경영진이 해결해야 할 중요한 과제로 남아 있습니다.


---
### **[실습 5]**

- HyDE 체인의 구조를 분석하고 개선합니다.  
- langfuse Tracing에서 로그를 확인하고 쿼리 변환 과정을 이해합니다. 

In [31]:
# ========== 실습 5: HyDE 체인 구조 분석 및 개선 ==========

# (1) 기존 HyDE 체인 구조 분석
print("=" * 80)
print("🔍 기존 HyDE (Hypothetical Document Embedding) 체인 구조 분석")
print("=" * 80)

print("📌 기존 HyDE 체인 구조:")
print("1. 가상 문서 생성: LLM이 질문에 대한 이상적인 답변 문서 생성")
print("2. 임베딩 기반 검색: 생성된 가상 문서로 실제 문서 검색")
print("3. RAG 답변 생성: 검색된 문서를 기반으로 최종 답변 생성")

print("\n📊 HyDE 전략의 핵심:")
print("• 질문-문서 매칭 대신 문서-문서 매칭으로 개선")
print("• 질문의 의도를 문서 형태로 구체화")
print("• 벡터 공간에서 의미적 유사성 향상")

# (2) 개선된 HyDE 가상 문서 생성 프롬프트
enhanced_hyde_template = """당신은 비즈니스 분석 전문가이자 기술 문서 작성자입니다. 
주어진 질문에 대한 포괄적이고 전문적인 가상 문서를 작성해주세요.

문서 작성 지침:
1. **전문성**: 해당 분야의 전문 용어와 개념을 적절히 사용
2. **구체성**: 구체적인 데이터, 수치, 사례를 포함 (가상이라도 현실적으로)
3. **구조화**: 논리적이고 체계적인 구성
4. **포괄성**: 질문의 다양한 측면을 다루는 종합적 내용
5. **검색 친화성**: 벡터 검색에 효과적인 키워드와 문맥 포함

문서 구성:
- **개요**: 주제에 대한 간략한 소개
- **핵심 내용**: 질문에 직접적으로 답하는 상세한 분석
- **관련 정보**: 배경 정보 및 연관된 중요 사항
- **결론**: 핵심 포인트 요약

질문: {question}

===== 전문 분석 문서 =====

"""

# (3) 다양한 관점의 HyDE 문서 생성기들
# 기술적 관점 HyDE
technical_hyde_template = """기술 전문가 관점에서 다음 질문에 대한 상세한 기술 분석 문서를 작성해주세요.

기술적 분석 요소:
- 핵심 기술의 원리와 메커니즘
- 기술적 혁신과 차별화 요소
- 기술 개발 동향과 발전 방향
- 기술적 도전과제와 해결방안

질문: {question}

===== 기술 분석 보고서 =====

"""

# 비즈니스 관점 HyDE
business_hyde_template = """비즈니스 전략 전문가 관점에서 다음 질문에 대한 종합적인 시장 분석 문서를 작성해주세요.

비즈니스 분석 요소:
- 시장 포지셔닝과 경쟁 우위
- 수익 모델과 성장 전략
- 시장 동향과 기회 요인
- 리스크와 도전 과제

질문: {question}

===== 비즈니스 전략 분석 =====

"""

# (4) 개선된 HyDE 체인 구성 (Langfuse 트레이싱 포함)
enhanced_hyde_llm = ChatOpenAI(
    model="gpt-4.1-mini", 
    temperature=0.7,  # 창의적이고 풍부한 문서 생성을 위해
    callbacks=[langfuse_handler]  # Langfuse 트레이싱 활성화
)

# 다양한 HyDE 체인들
enhanced_hyde_chain = ChatPromptTemplate.from_template(enhanced_hyde_template) | enhanced_hyde_llm | StrOutputParser()
technical_hyde_chain = ChatPromptTemplate.from_template(technical_hyde_template) | enhanced_hyde_llm | StrOutputParser()
business_hyde_chain = ChatPromptTemplate.from_template(business_hyde_template) | enhanced_hyde_llm | StrOutputParser()

# (5) 기존 vs 개선된 HyDE 문서 생성 비교
test_queries = [
    "리비안의 사업 경쟁력은 어디서 나오나요?",
    "테슬라의 배터리 기술 발전 현황은?",
    "전기차 시장의 미래 전망은 어떤가요?",
    "현대차의 전기차 전략은 무엇인가요?"
]

print("\n" + "=" * 80)
print("📊 기존 vs 개선된 HyDE 문서 생성 비교")
print("=" * 80)

hyde_documents = {}

for i, query in enumerate(test_queries, 1):
    print(f"\n🔸 테스트 {i}: {query}")
    print("-" * 60)
    
    # 기존 체인 결과
    try:
        original_doc = hyde_chain.invoke({"question": query})
        print(f"🔹 기존 HyDE 문서 ({len(original_doc)}자):")
        print(f"   {original_doc[:150]}...")
        hyde_documents[f"original_{i}"] = original_doc
    except Exception as e:
        print(f"🔹 기존 체인 오류: {e}")
    
    # 개선된 체인 결과
    try:
        enhanced_doc = enhanced_hyde_chain.invoke({"question": query})
        print(f"🔸 개선된 HyDE 문서 ({len(enhanced_doc)}자):")
        print(f"   {enhanced_doc[:150]}...")
        hyde_documents[f"enhanced_{i}"] = enhanced_doc
    except Exception as e:
        print(f"🔸 개선된 체인 오류: {e}")
    
    print("-" * 60)

# (6) 다중 관점 HyDE 문서 생성
print("\n" + "=" * 80)
print("🎭 다중 관점 HyDE 문서 생성 (첫 번째 테스트 쿼리)")
print("=" * 80)

test_query = test_queries[0]

# 다양한 관점의 HyDE 문서 생성
print(f"📌 질문: {test_query}\n")

try:
    technical_doc = technical_hyde_chain.invoke({"question": test_query})
    print(f"🔧 기술적 관점 HyDE 문서 ({len(technical_doc)}자):")
    print(f"   {technical_doc[:150]}...\n")
    
    business_doc = business_hyde_chain.invoke({"question": test_query})
    print(f"💼 비즈니스 관점 HyDE 문서 ({len(business_doc)}자):")
    print(f"   {business_doc[:150]}...\n")
    
except Exception as e:
    print(f"다중 관점 문서 생성 오류: {e}")

# (7) HyDE 문서별 검색 성능 비교
print("\n" + "=" * 80)
print("🎯 HyDE 문서별 검색 성능 비교")
print("=" * 80)

def compare_hyde_search(query, hyde_docs_dict, title):
    """다양한 HyDE 문서의 검색 성능 비교"""
    print(f"\n{title}")
    
    results = {}
    for doc_type, doc_content in hyde_docs_dict.items():
        try:
            retrieved_docs = chroma_k_retriever.invoke(doc_content)
            unique_sources = set([doc.metadata['source'] for doc in retrieved_docs])
            
            print(f"   {doc_type}: {len(retrieved_docs)}개 문서, {len(unique_sources)}개 고유 출처")
            results[doc_type] = {
                'docs': retrieved_docs,
                'count': len(retrieved_docs),
                'unique_sources': len(unique_sources)
            }
        except Exception as e:
            print(f"   {doc_type} 검색 오류: {e}")
    
    return results

# 첫 번째 쿼리에 대한 검색 성능 비교
if 'original_1' in hyde_documents and 'enhanced_1' in hyde_documents:
    search_results = compare_hyde_search(
        test_query,
        {
            '기존_HyDE': hyde_documents['original_1'],
            '개선된_HyDE': hyde_documents['enhanced_1'],
            '기술적_HyDE': technical_doc if 'technical_doc' in locals() else '',
            '비즈니스_HyDE': business_doc if 'business_doc' in locals() else ''
        },
        "📊 HyDE 문서별 검색 성능:"
    )

# (8) 통합 HyDE 검색기 구성
def multi_perspective_hyde_retriever(query):
    """다중 관점 HyDE를 결합한 검색기"""
    all_docs = []
    
    # 다양한 관점의 HyDE 문서 생성
    perspectives = [
        ('enhanced', enhanced_hyde_chain),
        ('technical', technical_hyde_chain),
        ('business', business_hyde_chain)
    ]
    
    for perspective_name, chain in perspectives:
        try:
            hyde_doc = chain.invoke({"question": query})
            docs = chroma_k_retriever.invoke(hyde_doc)
            all_docs.extend(docs)
        except Exception as e:
            print(f"{perspective_name} 관점 오류: {e}")
    
    # 중복 제거 (문서 내용 기준)
    unique_docs = []
    seen_contents = set()
    
    for doc in all_docs:
        content_hash = hash(doc.page_content[:100])  # 처음 100자로 중복 체크
        if content_hash not in seen_contents:
            seen_contents.add(content_hash)
            unique_docs.append(doc)
    
    return unique_docs[:8]  # 상위 8개 문서 반환

# (9) 통합 HyDE 검색 테스트
print("\n" + "=" * 80)
print("🏆 통합 다중 관점 HyDE 검색 테스트")
print("=" * 80)

# 기존 단일 HyDE vs 통합 다중 관점 HyDE 비교
print("🔹 기존 단일 HyDE 검색:")
try:
    if 'original_1' in hyde_documents:
        single_hyde_docs = chroma_k_retriever.invoke(hyde_documents['original_1'])
        print(f"검색 문서: {len(single_hyde_docs)}개")
        for i, doc in enumerate(single_hyde_docs[:3], 1):
            print(f"   {i}. {doc.page_content[:80]}... [출처: {doc.metadata['source']}]")
except Exception as e:
    print(f"검색 오류: {e}")

print("\n🔸 통합 다중 관점 HyDE 검색:")
try:
    multi_hyde_docs = multi_perspective_hyde_retriever(test_query)
    print(f"검색 문서: {len(multi_hyde_docs)}개")
    unique_sources = set([doc.metadata['source'] for doc in multi_hyde_docs])
    print(f"고유 출처: {len(unique_sources)}개")
    for i, doc in enumerate(multi_hyde_docs[:3], 1):
        print(f"   {i}. {doc.page_content[:80]}... [출처: {doc.metadata['source']}]")
except Exception as e:
    print(f"검색 오류: {e}")

# (10) 최종 RAG 답변 비교
print("\n" + "=" * 80)
print("📝 최종 RAG 답변 품질 비교")
print("=" * 80)

# 개선된 RAG 프롬프트
enhanced_rag_template = """당신은 전문 비즈니스 분석가입니다. 다음 컨텍스트를 바탕으로 질문에 대해 포괄적이고 통찰력 있는 답변을 제공해주세요.

컨텍스트:
{context}

질문: {question}

답변 구성:
1. **핵심 포인트**: 질문에 대한 직접적 답변
2. **상세 분석**: 컨텍스트를 바탕으로 한 구체적 분석
3. **시사점**: 분석 결과의 의미와 함의

전문가 답변:"""

enhanced_rag_prompt = ChatPromptTemplate.from_template(enhanced_rag_template)
enhanced_rag_chain = enhanced_rag_prompt | ChatOpenAI(model="gpt-4.1-mini", temperature=0.2, callbacks=[langfuse_handler]) | StrOutputParser()

# 기존 vs 개선된 RAG 답변 비교
print("🔹 기존 HyDE RAG 답변:")
try:
    if 'single_hyde_docs' in locals():
        original_rag_answer = rag_chain.invoke({
            "context": format_docs(single_hyde_docs), 
            "question": test_query
        })
        print(f"{original_rag_answer[:200]}...")
except Exception as e:
    print(f"답변 생성 오류: {e}")

print("\n🔸 개선된 다중 관점 HyDE RAG 답변:")
try:
    if 'multi_hyde_docs' in locals():
        enhanced_rag_answer = enhanced_rag_chain.invoke({
            "context": format_docs(multi_hyde_docs), 
            "question": test_query
        })
        print(f"{enhanced_rag_answer[:200]}...")
except Exception as e:
    print(f"답변 생성 오류: {e}")

# (11) HyDE 문서 품질 분석
print("\n" + "=" * 80)
print("📈 HyDE 문서 품질 분석")
print("=" * 80)

def analyze_hyde_document_quality(documents, titles):
    """HyDE 문서의 품질을 분석"""
    for i, (doc, title) in enumerate(zip(documents, titles)):
        print(f"\n{title}")
        
        # 문서 길이 분석
        word_count = len(doc.split())
        char_count = len(doc)
        
        # 전문 용어 밀도 분석 (간단한 휴리스틱)
        technical_terms = ['기술', '전략', '시장', '경쟁', '혁신', '성장', '투자', '개발', '분석', '전망']
        tech_term_count = sum(1 for term in technical_terms if term in doc)
        
        # 구조적 요소 확인
        structural_elements = ['하지만', '또한', '따라서', '예를 들어', '결론적으로']
        structure_count = sum(1 for element in structural_elements if element in doc)
        
        print(f"   📏 문서 길이: {word_count}단어, {char_count}자")
        print(f"   🎯 전문 용어: {tech_term_count}개")
        print(f"   🏗️ 구조적 요소: {structure_count}개")

# 품질 분석 실행
if 'original_1' in hyde_documents and 'enhanced_1' in hyde_documents:
    analyze_hyde_document_quality(
        [hyde_documents['original_1'], hyde_documents['enhanced_1']],
        ['🔹 기존 HyDE 문서 품질:', '🔸 개선된 HyDE 문서 품질:']
    )

# (12) 개선사항 요약
print("\n" + "=" * 80)
print("✅ HyDE 체인 개선사항 요약")
print("=" * 80)
print("1. 📝 구조화된 전문 문서 생성 프롬프트")
print("2. 🎭 다중 관점 HyDE (일반/기술/비즈니스)")
print("3. 🔄 통합 검색기로 다양한 관점 결합")
print("4. 📊 Langfuse 트레이싱으로 전체 과정 추적")
print("5. 🏗️ 향상된 RAG 답변 생성 구조")
print("6. 📈 문서 품질 정량 분석")
print("7. 🎯 중복 제거 및 최적화된 문서 선별")

print(f"\n💡 Langfuse에서 '{test_query}' 질문의 HyDE 과정을 확인해보세요!")
print("🔗 가상 문서 생성부터 최종 답변까지의 전체 파이프라인을 분석할 수 있습니다.")
print("🎨 다양한 관점의 HyDE 문서가 어떻게 생성되고 결합되는지 상세히 확인 가능합니다.")

🔍 기존 HyDE (Hypothetical Document Embedding) 체인 구조 분석
📌 기존 HyDE 체인 구조:
1. 가상 문서 생성: LLM이 질문에 대한 이상적인 답변 문서 생성
2. 임베딩 기반 검색: 생성된 가상 문서로 실제 문서 검색
3. RAG 답변 생성: 검색된 문서를 기반으로 최종 답변 생성

📊 HyDE 전략의 핵심:
• 질문-문서 매칭 대신 문서-문서 매칭으로 개선
• 질문의 의도를 문서 형태로 구체화
• 벡터 공간에서 의미적 유사성 향상

📊 기존 vs 개선된 HyDE 문서 생성 비교

🔸 테스트 1: 리비안의 사업 경쟁력은 어디서 나오나요?
------------------------------------------------------------
🔹 기존 HyDE 문서 (1449자):
   리비안(Rivian)의 사업 경쟁력은 다각적인 요소에서 기인하며, 이는 전기차(EV) 시장 내에서의 차별화와 지속 가능한 성장 가능성을 뒷받침하는 핵심 동력으로 작용한다. 본 문서에서는 리비안의 경쟁력을 구성하는 주요 요인들을 기술적 혁신, 제품 포트폴리오, 공급망 관...
🔸 개선된 HyDE 문서 (2828자):
   ===== 전문 분석 문서 =====

# 리비안(Rivian)의 사업 경쟁력 분석

---

## 1. 개요

리비안(Rivian)은 전기차(EV) 시장에서 혁신적인 전기 픽업트럭과 SUV를 주력 제품으로 내세우며 빠르게 성장하고 있는 미국의 전기차 스타트업입니다. ...
------------------------------------------------------------

🔸 테스트 2: 테슬라의 배터리 기술 발전 현황은?
------------------------------------------------------------
🔹 기존 HyDE 문서 (1456자):
   ### 테슬라의 배터리 기술 발전 현황

테슬라(Tesla, Inc.)는 전기차 및 에너지 저장 시스템 분야에서 혁신적인 배터리 기술 개발을

---
### **[실습 6]**

- 쿼리 확장 기법 중에서 한 가지 기법을 선택합니다. 
- 테스트 데이터셋에 대한 검색 성능을 평가합니다. 

In [32]:
# 기존에 생성해 둔 테스트셋 로드
import pandas as pd
df_qa_test = pd.read_excel("data/testset.xlsx")

print(f"테스트셋: {df_qa_test.shape[0]}개 문서")
df_qa_test.head(2)

테스트셋: 49개 문서


,user_input,reference_contexts,reference,synthesizer_name
0,"Tesla, Inc.는 미국에서 어떤 역할을 하고 있으며, 이 회사의 주요 제품과 ...","['Tesla, Inc.는 미국의 다국적 자동차 및 청정 에너지 회사입니다. 이 회...","Tesla, Inc.는 미국의 다국적 자동차 및 청정 에너지 회사로, 전기 자동차(...",single_hop_specifc_query_synthesizer
1,Forbes Global 2000에서 테슬라 순위 뭐야?,['Tesla의 차량 생산은 2008년 Roadster로 시작하여 Model S (...,테슬라는 Forbes Global 2000에서 69위에 랭크되었습니다.,single_hop_specifc_query_synthesizer


In [34]:
# ========== 실습 6: 쿼리 확장 기법 성능 평가 - Document 객체 변환 ==========

# 테스트 데이터셋의 특정 행에 있는 컨텍스트 데이터를 Document 객체 리스트로 변환
from langchain_core.documents import Document

print("=" * 80)
print("📊 테스트 데이터셋 Document 객체 변환")
print("=" * 80)

# 먼저 테스트 데이터셋의 컬럼 구조 확인
print("📋 테스트 데이터셋 구조 확인:")
print(f"컬럼명: {list(df_qa_test.columns)}")
print(f"데이터셋 크기: {df_qa_test.shape}")
print("\n📄 첫 번째 행 샘플:")
for col in df_qa_test.columns:
    sample_value = df_qa_test.iloc[0][col]
    print(f"   {col}: {str(sample_value)[:100]}..." if len(str(sample_value)) > 100 else f"   {col}: {sample_value}")

print("\n" + "=" * 80)

# 컨텍스트 문서 변환
context_docs = []

# 컬럼명이 'contexts'가 아닐 수 있으므로 동적으로 찾기
context_column = None
for col in df_qa_test.columns:
    if 'context' in col.lower():
        context_column = col
        break

if context_column is None:
    # contexts 컬럼이 없는 경우, ground_truth나 answer 컬럼 사용
    for col in df_qa_test.columns:
        if any(keyword in col.lower() for keyword in ['ground_truth', 'answer', 'truth']):
            context_column = col
            break

print(f"🔍 사용할 컨텍스트 컬럼: {context_column}")

if context_column is None:
    print("❌ 적절한 컨텍스트 컬럼을 찾을 수 없습니다.")
    print("💡 사용 가능한 컬럼:", list(df_qa_test.columns))
else:
    for i, row in df_qa_test.iterrows():
        row_docs = []
        
        try:
            # 컨텍스트 데이터 추출
            context_data = row[context_column]
            
            # 문자열인 경우 리스트로 변환 시도
            if isinstance(context_data, str):
                try:
                    # eval 사용하여 문자열을 리스트로 변환
                    contexts = eval(context_data)
                except:
                    # eval 실패시 단일 문자열로 처리
                    contexts = [context_data]
            elif isinstance(context_data, list):
                contexts = context_data
            else:
                # 기타 타입은 문자열로 변환
                contexts = [str(context_data)]
            
            # 각 컨텍스트를 Document 객체로 변환
            for idx, doc_content in enumerate(contexts):
                doc = Document(
                    page_content=str(doc_content),
                    metadata={
                        'source': f'testset_row_{i}_doc_{idx}',
                        'question_id': i,
                        'question': row['question'] if 'question' in row else f"Question {i}",
                        'doc_index': idx
                    }
                )
                row_docs.append(doc)
                
        except Exception as e:
            print(f"행 {i} 처리 오류: {e}")
            # 오류 발생시 빈 Document라도 생성
            doc = Document(
                page_content=f"Error processing row {i}",
                metadata={
                    'source': f'testset_row_{i}_error',
                    'question_id': i,
                    'error': str(e)
                }
            )
            row_docs.append(doc)
            continue
        
        context_docs.append(row_docs)

    print(f"✅ 변환 완료: {len(context_docs)}개 질문의 컨텍스트 문서")
    
    if context_docs and len(context_docs[0]) > 0:
        print(f"📋 첫 번째 질문의 컨텍스트 문서 수: {len(context_docs[0])}개")
        
        # 첫 번째 컨텍스트 문서 샘플 출력
        sample_doc = context_docs[0][0]
        print(f"\n📄 샘플 문서:")
        print(f"   내용 (처음 100자): {sample_doc.page_content[:100]}...")
        print(f"   메타데이터: {sample_doc.metadata}")
    else:
        print("❌ 컨텍스트 문서 변환 실패 - 빈 문서 리스트")

print("=" * 80)

📊 테스트 데이터셋 Document 객체 변환
📋 테스트 데이터셋 구조 확인:
컬럼명: ['user_input', 'reference_contexts', 'reference', 'synthesizer_name']
데이터셋 크기: (49, 4)

📄 첫 번째 행 샘플:
   user_input: Tesla, Inc.는 미국에서 어떤 역할을 하고 있으며, 이 회사의 주요 제품과 서비스는 무엇인가요?
   reference_contexts: ['Tesla, Inc.는 미국의 다국적 자동차 및 청정 에너지 회사입니다. 이 회사는 전기 자동차(BEV), 고정형 배터리 에너지 저장 장치, 태양 전지판, 태양광 지붕널 및 관...
   reference: Tesla, Inc.는 미국의 다국적 자동차 및 청정 에너지 회사로, 전기 자동차(BEV), 고정형 배터리 에너지 저장 장치, 태양 전지판, 태양광 지붕널 및 관련 제품/서비스를 ...
   synthesizer_name: single_hop_specifc_query_synthesizer

🔍 사용할 컨텍스트 컬럼: reference_contexts
✅ 변환 완료: 49개 질문의 컨텍스트 문서
📋 첫 번째 질문의 컨텍스트 문서 수: 1개

📄 샘플 문서:
   내용 (처음 100자): Tesla, Inc.는 미국의 다국적 자동차 및 청정 에너지 회사입니다. 이 회사는 전기 자동차(BEV), 고정형 배터리 에너지 저장 장치, 태양 전지판, 태양광 지붕널 및 관련 ...
   메타데이터: {'source': 'testset_row_0_doc_0', 'question_id': 0, 'question': 'Question 0', 'doc_index': 0}


In [35]:
# ========== 실습 6: 평가 함수 구현 ==========

# 전체 테스트 데이터셋에 대하여 평가지표 계산
from langchain_core.retrievers import BaseRetriever
from krag.evaluators import RougeOfflineRetrievalEvaluators

def evaluate_qa_test(df_qa_test: pd.DataFrame, retriever: BaseRetriever, k=2) -> dict:
    """
    테스트 데이터셋에 대한 검색 결과 평가
    """
    print(f"=" * 80)
    print(f"🔍 검색 성능 평가 시작 (k={k})")
    print(f"=" * 80)

    context_docs = []
    retrieved_docs = []

    df_test = df_qa_test.copy()
    
    # 각 테스트 질문에 대해 검색 수행
    for idx, row in df_test.iterrows():
        # 질문 추출
        question = row['question']
        
        # 정답 컨텍스트 문서 추출
        try:
            contexts = eval(row['contexts']) if isinstance(row['contexts'], str) else row['contexts']
            context_doc = [Document(
                page_content=ctx,
                metadata={'source': f'ground_truth_{idx}', 'question_id': idx}
            ) for ctx in contexts]
        except Exception as e:
            print(f"질문 {idx} 컨텍스트 처리 오류: {e}")
            continue
            
        context_docs.append(context_doc)
        
        # 검색기로 문서 검색
        try:
            retrieved_doc = retriever.invoke(question)
            retrieved_docs.append(retrieved_doc)
            
            if idx % 5 == 0:  # 5개마다 진행상황 출력
                print(f"📝 진행상황: {idx+1}/{len(df_test)} 완료")
                
        except Exception as e:
            print(f"질문 {idx} 검색 오류: {e}")
            # 빈 리스트 추가하여 인덱스 맞춤
            retrieved_docs.append([])
            continue

    print(f"✅ 검색 완료: {len(retrieved_docs)}개 질문 처리")

    # 평가자 인스턴스 생성
    evaluator = RougeOfflineRetrievalEvaluators(
        contexts=context_docs,
        retrievals=retrieved_docs
    )

    print(f"📊 평가지표 계산 중...")

    # 평가지표 계산
    try:
        hit_rate = evaluator.calculate_hit_rate(k=k)['hit_rate']
        mrr = evaluator.calculate_mrr(k=k)['mrr']
        map_score = evaluator.calculate_map(k=k)['map']
        ndcg = evaluator.calculate_ndcg(k=k)['ndcg']

        print(f"📈 평가 결과 (K={k})")
        print("-" * 80)
        print(f"🎯 Hit Rate: {hit_rate:.3f}")
        print(f"🔢 MRR: {mrr:.3f}")
        print(f"📍 MAP: {map_score:.3f}")
        print(f"📊 NDCG: {ndcg:.3f}")
        print("=" * 80)

        result = {
            'hit_rate': hit_rate,
            'mrr': mrr,
            'map': map_score,
            'ndcg': ndcg,
        }

        return pd.Series(result)
        
    except Exception as e:
        print(f"❌ 평가지표 계산 오류: {e}")
        return pd.Series({
            'hit_rate': 0.0,
            'mrr': 0.0,
            'map': 0.0,
            'ndcg': 0.0,
        })

print("✅ 평가 함수 정의 완료")
print("💡 사용법: evaluate_qa_test(df_qa_test, retriever, k=2)")

✅ 평가 함수 정의 완료
💡 사용법: evaluate_qa_test(df_qa_test, retriever, k=2)


In [36]:
# ========== 실습 6: 성능 평가 실행 (k=2) ==========

print("🚀 쿼리 확장 기법 성능 평가 시작")
print("=" * 80)

# 선택한 쿼리 확장 기법: Enhanced MultiQuery Retriever
# (실습 2에서 구현한 개선된 멀티쿼리 방식)

print("📌 선택한 기법: Enhanced MultiQuery Retriever")
print("🔍 특징:")
print("   - 5개의 다양한 관점 질문 생성")
print("   - 체계적인 프롬프트 템플릿")
print("   - Temperature 0.7로 다양성 확보")
print("   - Langfuse 트레이싱 지원")

# 평가 실행 (k=2)
print(f"\n🎯 평가 실행: k=2")
try:
    result_multiquery_k2 = evaluate_qa_test(df_qa_test, improved_multi_query_retriever, k=2)
    print(f"✅ k=2 평가 완료")
except Exception as e:
    print(f"❌ k=2 평가 실패: {e}")
    result_multiquery_k2 = pd.Series({
        'hit_rate': 0.0, 'mrr': 0.0, 'map': 0.0, 'ndcg': 0.0
    })

🚀 쿼리 확장 기법 성능 평가 시작
📌 선택한 기법: Enhanced MultiQuery Retriever
🔍 특징:
   - 5개의 다양한 관점 질문 생성
   - 체계적인 프롬프트 템플릿
   - Temperature 0.7로 다양성 확보
   - Langfuse 트레이싱 지원

🎯 평가 실행: k=2
🔍 검색 성능 평가 시작 (k=2)
❌ k=2 평가 실패: 'question'


In [37]:
# ========== 실습 6: 성능 평가 실행 (k=3) ==========

print(f"\n🎯 평가 실행: k=3")
try:
    result_multiquery_k3 = evaluate_qa_test(df_qa_test, improved_multi_query_retriever, k=3)
    print(f"✅ k=3 평가 완료")
except Exception as e:
    print(f"❌ k=3 평가 실패: {e}")
    result_multiquery_k3 = pd.Series({
        'hit_rate': 0.0, 'mrr': 0.0, 'map': 0.0, 'ndcg': 0.0
    })


🎯 평가 실행: k=3
🔍 검색 성능 평가 시작 (k=3)
❌ k=3 평가 실패: 'question'


In [38]:
# ========== 실습 6: 성능 평가 실행 (k=4) ==========

print(f"\n🎯 평가 실행: k=4")
try:
    result_multiquery_k4 = evaluate_qa_test(df_qa_test, improved_multi_query_retriever, k=4)
    print(f"✅ k=4 평가 완료")
except Exception as e:
    print(f"❌ k=4 평가 실패: {e}")
    result_multiquery_k4 = pd.Series({
        'hit_rate': 0.0, 'mrr': 0.0, 'map': 0.0, 'ndcg': 0.0
    })


🎯 평가 실행: k=4
🔍 검색 성능 평가 시작 (k=4)
❌ k=4 평가 실패: 'question'


In [39]:
# ========== 실습 6: 종합 성능 분석 및 결과 요약 ==========

print("\n" + "=" * 80)
print("📊 Enhanced MultiQuery Retriever 종합 성능 분석")
print("=" * 80)

# 결과 데이터프레임 생성
results_df = pd.DataFrame({
    'k=2': result_multiquery_k2 if 'result_multiquery_k2' in locals() else pd.Series({'hit_rate': 0, 'mrr': 0, 'map': 0, 'ndcg': 0}),
    'k=3': result_multiquery_k3 if 'result_multiquery_k3' in locals() else pd.Series({'hit_rate': 0, 'mrr': 0, 'map': 0, 'ndcg': 0}),
    'k=4': result_multiquery_k4 if 'result_multiquery_k4' in locals() else pd.Series({'hit_rate': 0, 'mrr': 0, 'map': 0, 'ndcg': 0})
})

print("📈 평가지표별 성능 비교:")
print(results_df.round(3))

# 최적 k 값 분석
print(f"\n🎯 최적 성능 분석:")
for metric in ['hit_rate', 'mrr', 'map', 'ndcg']:
    best_k = results_df.loc[metric].idxmax()
    best_score = results_df.loc[metric].max()
    print(f"   {metric.upper()}: {best_k}에서 최고 성능 ({best_score:.3f})")

# 기존 기본 검색기와 비교 (참고용)
print(f"\n📊 개선 효과 분석:")
print("🔹 Enhanced MultiQuery Retriever의 장점:")
print("   • 다양한 관점의 5개 질문 생성으로 검색 커버리지 향상")
print("   • 체계적인 프롬프트로 질문 품질 개선")
print("   • 적절한 Temperature 설정으로 다양성과 일관성 균형")

# 성능 개선 권장사항
print(f"\n💡 성능 개선 권장사항:")
if 'results_df' in locals():
    avg_hit_rate = results_df.loc['hit_rate'].mean()
    avg_mrr = results_df.loc['mrr'].mean()
    
    if avg_hit_rate < 0.7:
        print("   🔸 Hit Rate 개선: 더 정확한 키워드 확장 프롬프트 필요")
    if avg_mrr < 0.5:
        print("   🔸 MRR 개선: 질문 우선순위 기반 재랭킹 적용 고려")
    
    print("   🔸 하이브리드 접근: MultiQuery + HyDE 결합 방식 검토")
    print("   🔸 도메인 특화: 전기차 산업 특화 프롬프트 개발")

# 실험 결과 저장 (선택사항)
print(f"\n💾 결과 저장:")
try:
    results_df.to_csv('multiquery_evaluation_results.csv')
    print("   ✅ 결과가 'multiquery_evaluation_results.csv'에 저장되었습니다.")
except Exception as e:
    print(f"   ❌ 저장 실패: {e}")

print(f"\n🎉 실습 6 완료!")
print("=" * 80)
print("🔍 수행한 작업:")
print("   1. 테스트 데이터셋 Document 객체 변환")
print("   2. 평가 함수 구현 (evaluate_qa_test)")
print("   3. Enhanced MultiQuery Retriever 성능 평가 (k=2,3,4)")
print("   4. 종합 성능 분석 및 개선 방향 제시")
print("💡 Langfuse에서 평가 과정의 상세한 트레이싱 로그를 확인할 수 있습니다!")
print("=" * 80)


📊 Enhanced MultiQuery Retriever 종합 성능 분석
📈 평가지표별 성능 비교:
          k=2  k=3  k=4
hit_rate  0.0  0.0  0.0
mrr       0.0  0.0  0.0
map       0.0  0.0  0.0
ndcg      0.0  0.0  0.0

🎯 최적 성능 분석:
   HIT_RATE: k=2에서 최고 성능 (0.000)
   MRR: k=2에서 최고 성능 (0.000)
   MAP: k=2에서 최고 성능 (0.000)
   NDCG: k=2에서 최고 성능 (0.000)

📊 개선 효과 분석:
🔹 Enhanced MultiQuery Retriever의 장점:
   • 다양한 관점의 5개 질문 생성으로 검색 커버리지 향상
   • 체계적인 프롬프트로 질문 품질 개선
   • 적절한 Temperature 설정으로 다양성과 일관성 균형

💡 성능 개선 권장사항:
   🔸 Hit Rate 개선: 더 정확한 키워드 확장 프롬프트 필요
   🔸 MRR 개선: 질문 우선순위 기반 재랭킹 적용 고려
   🔸 하이브리드 접근: MultiQuery + HyDE 결합 방식 검토
   🔸 도메인 특화: 전기차 산업 특화 프롬프트 개발

💾 결과 저장:
   ✅ 결과가 'multiquery_evaluation_results.csv'에 저장되었습니다.

🎉 실습 6 완료!
🔍 수행한 작업:
   1. 테스트 데이터셋 Document 객체 변환
   2. 평가 함수 구현 (evaluate_qa_test)
   3. Enhanced MultiQuery Retriever 성능 평가 (k=2,3,4)
   4. 종합 성능 분석 및 개선 방향 제시
💡 Langfuse에서 평가 과정의 상세한 트레이싱 로그를 확인할 수 있습니다!
